In [1]:
#fetch command line argument (prepped model)
#%reset
import sys
import os
from copy import deepcopy
import pickle
import torch
import plotly.offline as py
import plotly.graph_objs as go


sys.path.insert(0, os.path.abspath('./prep_model_scripts/'))
sys.path.insert(0, os.path.abspath('./visualizer_scripts/'))
from visualizer_helper_functions import *
from contrast_helper_functions import *
from featureviz_helper_functions import *
from subgraph_helper_functions import *
from ablation_functions import *
from receptive_field import *
from dissected_Conv2d import *
from copy import deepcopy


py.init_notebook_mode(connected=True)   #jupyter notebook only

prepped_model_folder = 'alexnet_sparse'    #set this to a subfolder of prunned_models

full_prepped_model_folder = os.path.abspath('prepped_models/%s'%prepped_model_folder)

possible_models = os.listdir('prepped_models')
print('possible models to visualizer are:')
print(possible_models)

print('\nYou\'ve chosen to visualize %s'%prepped_model_folder)


sys.path.insert(0,'prepped_models/%s'%prepped_model_folder)

import prep_model_params_used as prep_model_params

params = {}
params['prepped_model'] = prepped_model_folder
params['prepped_model_path'] = full_prepped_model_folder
params['device'] = 'cuda:2'

possible models to visualizer are:
['mnist_resize', 'alexnet_old.tar.gz', 'mnist_old', '.keep', 'mnist', 'old', 'alexnet10', 'letter_mixed_not_trained', 'imagenet10_sparse', 'alexnet_sparse_test', '.DS_Store', 'alexnet_sparse_full_model_10classes', 'imagenet10', 'alexnet_sparse', 'alexnet_lucent', 'mnist.tgz', 'alexnet', 'letter_mixed', 'googlenet10_test', '._.DS_Store', 'alexnet_corrupted', 'alexnet.tar.gz']

You've chosen to visualize alexnet_sparse


In [2]:
#Parameters

#Non-GUI parameters

#deepviz
params['deepviz_param'] = None
params['deepviz_optim'] = None
params['deepviz_transforms'] = None
params['deepviz_image_size'] = prep_model_params.deepviz_image_size

#backend
params['cuda'] = prep_model_params.cuda    #use gpu acceleration when running model forward
params['input_image_directory'] = prep_model_params.input_img_path+'/'   #path to directory of imput images you want fed through the network
params['preprocess'] = prep_model_params.preprocess     #torchvision transfrom to pass input images through
params['label_file_path'] = prep_model_params.label_file_path
params['criterion'] = prep_model_params.criterion
params['rank_img_path'] = prep_model_params.rank_img_path
params['num_workers'] = prep_model_params.num_workers
params['seed'] = prep_model_params.seed
params['batch_size'] = prep_model_params.batch_size
#params['dynamic_act_cache_num'] = 4  #max number of input image activations 'dynamic_activations' will have simultaneously

 
#aesthetic 

params['node_size'] = 12
params['edge_size'] = 1
params['max_node_inputs'] = 10    #there is a dropdown showing the top weighted edge inputs to nodes, how many maps in dropdown?
params['layer_colors'] = ['rgba(31,119,180,', 
                          'rgba(255,127,14,',
                          'rgba(44,160,44,', 
                          'rgba(214,39,40,',
                          'rgba(39, 208, 214,', 
                          'rgba(242, 250, 17,',
                          'rgba(196, 94, 255,',
                          'rgba(193, 245, 5,',
                          'rgba(245, 85, 5,',
                          'rgba(5, 165, 245,',
                          'rgba(245, 5, 105,',
                          'rgba(218, 232, 23,',
                          'rgba(148, 23, 232,',
                          'rgba(23, 232, 166,',]




#GUI parameters initialization (these parameters can be set in the GUI, but what values should they be initialized to?)
target_category = 'overall'     #category of images edges and nodes are weighted based on (which subgraph) 
rank_type = 'actxgrad'       #weighting criterion (actxgrad, act, grad, or weight)
projection = 'MDS smooth'           #how nodes within a layer are projected into the 2d plane (MDS or Grid)
edge_threshold = [.7,1]     #what range do edge ranks need to be in to be visualized
node_threshold = [.4,1]     #only relevant for hierarchical subgraph 

#### DONT EDIT BELOW initializations

figure_init = go.Figure()
figure_init.add_trace(go.Scatter(
            x=[],
            y=[]))
figure_init.update_layout(xaxis=dict(visible=False),
                  yaxis=dict(visible=False),
                  annotations = [dict(text="No Inputs",
                                      xref="paper",
                                      yref="paper",
                                      showarrow=False,
                                      font=dict(size=28))]
                 )

params['max_edge_weight'] = 1  #for the edge threshold slider, this dynamically adjusted its max value to max edge rank
                     #before there were multiple rank criterions, which made things confusing
                     #so well just fix it to 1 for now

In [3]:
#load Model

model_dis = dissect_model(deepcopy(prep_model_params.model),store_ranks=True,clear_ranks=True,cuda=params['cuda'],device=params['device']) #version of model with accessible preadd activations in Conv2d modules 
if params['cuda']:
    model_dis.cuda()
model_dis = model_dis.eval()    
model_dis.to(params['device'])

print('loaded model:')
print(prep_model_params.model)
        
#del prep_model_params.model
model = prep_model_params.model
if params['cuda']:
    model.cuda()
model = model.eval()
model.to(params['device'])

loaded model:
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
#load misc graph data
print('loading misc graph data')
misc_data = pickle.load(open('./prepped_models/%s/misc_graph_data.pkl'%prepped_model_folder,'rb'))
params['layer_nodes'] = misc_data['layer_nodes']
params['num_layers'] = misc_data['num_layers']
params['num_nodes'] = misc_data['num_nodes']
params['categories'] = misc_data['categories']
params['num_img_chan'] = misc_data['num_img_chan']
params['imgnode_positions'] = misc_data['imgnode_positions']
params['imgnode_colors'] = misc_data['imgnode_colors']
params['imgnode_names'] = misc_data['imgnode_names']
params['prepped_model_path'] = full_prepped_model_folder
params['ranks_data_path'] = full_prepped_model_folder+'/ranks/'


print('model has categories:')
print(params['categories'])

loading misc graph data
model has categories:
['overall', 'Afghan_hound', 'African_chameleon', 'African_crocodile', 'African_elephant', 'African_grey', 'African_hunting_dog', 'Airedale', 'American_Staffordshire_terrier', 'American_alligator', 'American_black_bear', 'American_chameleon', 'American_coot', 'American_egret', 'American_lobster', 'Angora', 'Appenzeller', 'Arabian_camel', 'Arctic_fox', 'Australian_terrier', 'Band_Aid', 'Bedlington_terrier', 'Bernese_mountain_dog', 'Blenheim_spaniel', 'Border_collie', 'Border_terrier', 'Boston_bull', 'Bouvier_des_Flandres', 'Brabancon_griffon', 'Brittany_spaniel', 'CD_player', 'Cardigan_corgi', 'Chesapeake_Bay_retriever', 'Chihuahua', 'Christmas_stocking', 'Crock_Pot', 'Dandie_Dinmont', 'Doberman', 'Dungeness_crab', 'Dutch_oven', 'Egyptian_cat', 'English_foxhound', 'English_setter', 'English_springer', 'EntleBucher', 'Eskimo_dog', 'European_fire_salamander', 'European_gallinule', 'French_bulldog', 'French_horn', 'French_loaf', 'German_shepherd

In [5]:
#load nodes df
print('loading nodes rank data')
target_node = 'loss'

categories_nodes_df = pd.read_csv('prepped_models/%s/ranks/categories_nodes_ranks.csv'%prepped_model_folder)
target_nodes_df = categories_nodes_df.loc[categories_nodes_df['category']==target_category]

target_nodes_df = minmax_normalize_ranks_df(target_nodes_df,params,weight=False)

weight_nodes_df = pd.read_csv('prepped_models/%s/ranks/weight_nodes_ranks.csv'%prepped_model_folder)

weight_nodes_df = minmax_normalize_ranks_df(weight_nodes_df,params,weight=True)

node_colors,node_weights = gen_node_colors(target_nodes_df,rank_type,params) 

#load node positions
print('loading node position data')
all_node_positions = pickle.load(open('./prepped_models/%s/node_positions.pkl'%prepped_model_folder,'rb'))

if projection == 'Grid':
    node_positions = all_node_positions[projection]
else:
    node_positions = all_node_positions[projection][rank_type]



loading nodes rank data


/mnt/data/chris/dropbox/Research-Hamblin/Projects/cnn_subgraph_visualizer/visualizer_scripts/visualizer_helper_functions.py:146: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



loading node position data


In [6]:
#load edges
print('loading edge data')

categories_edges_df = None
if os.path.exists('prepped_models/%s/edge_ranks.csv'%prepped_model_folder):
    categories_edges_df = pd.read_csv('prepped_models/%s/ranks/categories_edges_ranks.csv'%prepped_model_folder)   #load edges

if categories_edges_df is not None:
    #overall_edges_df = categories_edges_df.loc[categories_edges_df['category']=='overall']
    target_edges_df = categories_edges_df.loc[categories_edges_df['category']==target_category]
else:
    #overall_edges_df = rank_file_2_df(os.path.join(params['ranks_data_path'],'categories_edges','overall_edges_rank.pt'))
    target_edges_df = rank_file_2_df(os.path.join(params['ranks_data_path'],'categories_edges','%s_edges_rank.pt'%target_category))

target_edges_df = minmax_normalize_ranks_df(target_edges_df,params,weight=False)

weight_edges_df = pd.read_csv('prepped_models/%s/ranks/weight_edges_ranks.csv'%prepped_model_folder)
  
weight_edges_df = minmax_normalize_ranks_df(weight_edges_df,params,weight=True)    
    
edges_thresholded_df = get_thresholded_ranksdf(edge_threshold,rank_type,target_edges_df)
 
    
num_edges = len(target_edges_df)
edges_df_columns = list(target_edges_df.columns)

edge_positions, edge_colors, edge_widths, edge_weights, edge_names, max_edge_width_indices = gen_edge_graphdata(edges_thresholded_df, node_positions, rank_type, target_category,params)

loading edge data


In [7]:
#Load Edge Kernels
print('loading convolutional kernels')
kernels = torch.load('prepped_models/%s/kernels.pt'%prepped_model_folder)

loading convolutional kernels


In [8]:
#Input Image names
params['input_image_directory'] = prep_model_params.input_img_path+'/'
params['input_image_list'] = os.listdir(params['input_image_directory'])
params['input_image_list'].sort()
input_image_name = params['input_image_list'][0]

In [9]:
receptive_fields = None
if os.path.exists('prepped_models/%s/receptive_fields.pkl'%prepped_model_folder):
    receptive_fields = pickle.load(open('prepped_models/%s/receptive_fields.pkl'%prepped_model_folder,'rb'))
    
input_image_size = 224   #got to figure out a way not to hard-code this
receptive_fields

OrderedDict([('features_-1',
              OrderedDict([('j', 1.0),
                           ('r', 1.0),
                           ('start', 0.5),
                           ('conv_stage', True),
                           ('output_shape', [-1, 3, 224, 224])])),
             ('features_0',
              OrderedDict([('j', 4.0),
                           ('r', 11.0),
                           ('start', 3.5),
                           ('input_shape', [-1, 3, 224, 224]),
                           ('output_shape', [-1, 64, 55, 55])])),
             ('features_1',
              OrderedDict([('j', 4.0),
                           ('r', 11.0),
                           ('start', 3.5),
                           ('input_shape', [-1, 64, 55, 55]),
                           ('output_shape', [-1, 64, 55, 55])])),
             ('features_2',
              OrderedDict([('j', 8.0),
                           ('r', 19.0),
                           ('start', 7.5),
                           

In [10]:
#Format Node Feature Maps
print('loading activation maps')

all_activations = {'nodes':{},'edges_in':{},'edges_out':{}}
if os.path.exists('prepped_models/%s/input_img_activations.pt'%prepped_model_folder):
    all_activations = torch.load('prepped_models/%s/input_img_activations.pt'%prepped_model_folder)


loading activation maps


In [11]:
#hidden state, stores python values within the html itself
state = {'projection':projection,'rank_type':rank_type,'edge_positions':edge_positions,'edge_colors': edge_colors, 'edge_widths':edge_widths,'edge_names':edge_names,
         'edge_threshold':edge_threshold,'edge_weights':edge_weights,'max_edge_width_indices':max_edge_width_indices,
         'node_positions':node_positions,'node_colors':node_colors,'node_weights':node_weights,'node_threshold':node_threshold,'target_category':target_category,'target_node':'loss',
         'node_select_history':['0'],'edge_select_history':[edge_names[0][0]],'last_trigger':None,'input_image_name':input_image_name,
         'imgnode_positions':params['imgnode_positions'],'imgnode_colors':params['imgnode_colors'],'imgnode_names':params['imgnode_names']}


In [12]:
#App Component Layouts
axis=dict(showbackground=False,
          showspikes=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          #range=[0,0],
          title=''
          )

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.00, y=-1.25, z=1.25)
)


network_graph_layout = go.Layout(
         #title="%s through Prunned Cifar10 CNN"%target_category,
         #title = target_category,
         #width=1000,
         clickmode = 'event+select',
         transition = {'duration': 20},
         height=500,
         #showlegend=False,
         margin = dict(l=20, r=20, t=20, b=20),
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
             aspectmode ="manual", 
             aspectratio = dict(x=1, y=0.5, z=0.5) #adjusting this stretches the network layer-to-layer
         ),
         scene_camera = camera,
         uirevision =  True   
         #hovermode='closest',
   )


input_image_layout = go.Layout(#width=200, 
                      #height=200,
                      uirevision = True,
                      margin=dict(
                        l=12,
                        r=1,
                        b=12,
                        t=1,
                        pad=10
                        ),
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)',
                        xaxis=dict(range=(0,10),showline=False,showgrid=False,showticklabels=False),
                        yaxis=dict(range=(0,10),showline=False,showgrid=False,showticklabels=False))


node_actmap_layout = go.Layout(
    #autosize=False,
    #width=270,
    #height=200,
    uirevision = True,
    margin=dict(
        l=1,
        r=1,
        b=1,
        t=1,
        pad=1
    ))


edge_inmap_layout = go.Layout(
    #title = 'edge input map',
    #autosize=False,
    #width=270,
    #height=200,
    uirevision = True,
    margin=dict(
        l=1,
        r=1,
        b=1,
        t=10,
        pad=1
    ))


edge_outmap_layout = go.Layout(
    #title = 'edge output map',
    #autosize=False,
    #width=270,
    #height=200,
    uirevision = True,
    margin=dict(
        l=1,
        r=1,
        b=1,
        t=10,
        pad=1
    ))


kernel_layout = go.Layout(
    #title='kernel'
    #autosize=False,
    #width=180,
    #height=120,
    uirevision = True,
    margin=dict(
        l=1,
        r=1,
        b=1,
        t=1,
        pad=1
    ))


In [13]:
#Generate Network Graph
combined_traces = gen_networkgraph_traces(state,params)
network_graph_fig=go.Figure(data=combined_traces, layout=network_graph_layout)

building graph from browser "state"


In [14]:
#Dash App Setup
print('setting up dash app')

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
from dash.exceptions import PreventUpdate
#import utils.dash_reusable_components as drc
import flask
import os

import json

from dash.dependencies import Input, Output, State

from plotly.subplots import make_subplots

from flask_caching import Cache

#external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(external_stylesheets = external_stylesheets)



if not os.path.exists(full_prepped_model_folder+'/cache/'):
    os.mkdir(full_prepped_model_folder+'/cache/')
CACHE_CONFIG = {
    # try 'filesystem' if you don't want to setup redis
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': full_prepped_model_folder+'/cache/'}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)
    


styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


theme =  {
    'dark': True,
    'detail': '#007439',
    'primary': '#00EA64',
    'secondary': '#6E6E6E',
}


app.layout = html.Div([
        html.Div(
            children = [
                
            html.Div(
                #Left side control panel
                children = [
                 html.Label('Subgraph Controls', style={'fontSize': 18,'font-weight':'bold'}),
                 html.Br(),
                 html.Label('Input'),
                 #dcc.Dropdown(
                 #  id='weight-category',
                 #  options=[{'label': i, 'value': i} for i in params['categories']],
                 #   value=target_category
                 #   ),
                dcc.Input(id='input-category',value=state['target_category']),
                html.Br(),
                html.Br(),
                html.Label('Output'),
                 #dcc.Dropdown(
                 #  id='weight-category',
                 #  options=[{'label': i, 'value': i} for i in params['categories']],
                 #   value=target_category
                 #   ),
                dcc.Dropdown(
                    id='target-node',
                    options=[
                    {'label': i, 'value': i} for i in ['loss']+[str(node) for node in list(range(params['num_nodes']))]
                    ],
                    value=state['target_node']),
                 html.Br(),
                 html.Label('Subgraph Criterion'),
                 dcc.Dropdown(
                    id='subgraph-criterion',
                    options=[
                        {'label': 'Activations*Grads', 'value': 'actxgrad'},
                        {'label': 'Activations', 'value': 'act'},
                        {'label': 'Gradients', 'value': 'grad'},
                        {'label': 'Weights', 'value': 'weight'},
                        {'label': 'Hierarchical', 'value': 'hierarchical'}
                        
                    ],
                    value='actxgrad'
                    ),
                 html.Br(),   
                 html.Label('Layer Projection'),
                 dcc.Dropdown(
                    id = 'layer-projection',
                    options=[
                        {'label': 'MDS', 'value': 'MDS'},
                        {'label': 'MDS smooth', 'value': 'MDS smooth'},
                        {'label': 'Grid', 'value': 'Grid'},
                        #{'label': 'SOM', 'value': 'SOM'}
                    ],
                    value='MDS smooth'
                    ),

                html.Br(),
                html.Label('Edge Thresholds'),
                    dcc.RangeSlider(
                        id='edge-thresh-slider',
                        min=0,
                        max=np.ceil(params['max_edge_weight']*10)/10,
                        step=0.001,
                        marks={i/10: str(i/10) for i in range(0,int(np.ceil(params['max_edge_weight']*10))+1,int(round(np.ceil(params['max_edge_weight']*10)/5)))},
                        value=edge_threshold,
                    ),
                html.Label('Node Thresholds'),
                    dcc.RangeSlider(
                        id='node-thresh-slider',
                        min=0,
                        max=1,
                        step=0.001,
                        marks={i/10: str(i/10.0) for i in range(0,11)},
                        value=node_threshold,
                    ),

                ], className="two columns",
                ),
                
            html.Div([
                dcc.Graph(
                    id='network-graph',
                    figure=network_graph_fig
                )
                ], className= 'ten columns'
                ),
            ], className="row"
        ),


                
        html.Div([
            html.Div([
            html.Label('Input Image', style={'fontSize': 18,'font-weight':'bold'}),
            dcc.Dropdown(id="dynamic-input-image-dropdown",value=params['input_image_list'][0]),
            #dcc.Dropdown(
            #    id='input-image-dropdown',
            #    options=[{'label': i, 'value': i} for i in params['input_image_list']+os.listdir(params['prepped_model_path']+'/visualizations/images/')],
            #    value=input_image_name
            #),
            html.Br(),
            dcc.Graph(
                id='img-actmap-graph',
                style={
               'width': '14vw',
               'height':'14vw'
                },
                figure=image2heatmap(params['input_image_directory']+input_image_name,input_image_layout),
                config={
                        'displayModeBar': False
                        }
            )
            ], className = "two columns"),

            html.Div([
            html.Label('Node', style={'fontSize': 18,'font-weight':'bold'}),
            dcc.Dropdown(
                id='node-actmap-dropdown',
                options=[{'label': str(j), 'value': str(j)} for j in params['imgnode_names']]+[{'label': str(i), 'value': str(i)} for i in range(params['num_nodes'])],
                value='0'
            ),
            html.Br(),
            dcc.Graph(
                id='node-actmap-graph',
                style={
               'width': '18vw',
               'height':'14vw'
                },
                figure=figure_init,
                config={
                        'displayModeBar': False
                        }
            ),
            dcc.Checklist(
                id = 'relu-checkbox',
                options = [{'label':'relu','value':'relu'}],
                value = []
                
            ),
            html.Div(id='node-sum', style={'whiteSpace': 'pre-line'}),
            html.Br(),
            html.Br(),
            dcc.Graph(
                id='node-deepviz-image',
                style={
               'width': '14vw',
               'height':'14vw'
                },
                figure=figure_init,
                config={
                        'displayModeBar': False
                        }
            )
            ], className = "three columns"),
            
            html.Div([
            html.Label('Node Inputs', style={'fontSize': 18,'font-weight':'bold'}),
            html.Br(),
            html.Div(dcc.Graph(
                id='node-inputs-graph',
                figure=figure_init,
                config={
                        'displayModeBar': False
                        }
            ),style={'overflowY': 'scroll', 'height': 500})
            ], className = "three columns"),

            html.Div([
            html.Label('Edge', style={'fontSize': 18,'font-weight':'bold'}),    
            dcc.Input(
                id='edge-actmaps-input',value=state['edge_names'][0][0], type='text'),
            #html.Button(id='edge-kernel-button',n_clicks=0, children='Submit'),
            html.Br(),
            html.Br(),
            html.Label('Kernel'),
            dcc.Graph(
                id='edge-kernel-graph',
                style={
               'width': '14vw',
               'height':'10vw'
                },
                figure=go.Figure(data=go.Heatmap(
                                    z = edgename_2_edge_figures(state['edge_names'][0][0], input_image_name, kernels, None,params)[0]),
                                 layout=kernel_layout
                                ),
                config={
                        'displayModeBar': False
                        }
            ),
            html.Br(),
            html.Br(),
            dcc.Graph(
               id='edge-deepviz-image',
               style={
              'width': '14vw',
              'height':'14vw'
               },
               figure=figure_init,
               config={
                       'displayModeBar': False
                       }
            )
            ], className = "two columns"),


            html.Div([
            html.Label('Edge Input'),
            html.Br(),
            dcc.Graph(
                id='edge-inmap-graph',
                style={
               'width': '18vw',
               'height':'14vw'
                },
                figure=figure_init,
                config={
                        'displayModeBar': False
                        }
            ),
            html.Div(id='edgein-sum', style={'whiteSpace': 'pre-line'}),
            html.Br(),
            html.Br(),
            html.Label('Edge Output'),
            html.Br(),
            dcc.Graph(
                id='edge-outmap-graph',
                style={
               'width': '18vw',
               'height':'14vw'
                },
                figure=figure_init,
                config={
                        'displayModeBar': False
                        }
            ),
            html.Div(id='edgeout-sum', style={'whiteSpace': 'pre-line'}),

            ], className = "two columns")


         ], className= 'row'
         ),
    
    
    html.Div([
            html.Div([
            html.Label('Image Manipulations', style={'fontSize': 18,'font-weight':'bold'}),
            html.Br(),
            html.Label('rotation'),
            dcc.Slider(
                id='image-rotation-slider',
                min=0,
                max=350,
                step=10,
                marks={
                        0:   '0°',
                        20:  '20°',
                        40:  '40°',
                        60:  '60°',
                        80:  '80°',
                        100: '100°',
                        120: '120°',
                        140: '140°',
                        160: '160°',
                        180: '180°',
                        200: '200°',
                        220: '220°',
                        240: '240°',
                        260: '260°',
                        280: '280°',
                        300: '300°',
                        320: '320°',
                        340: '240°',
                        },
                included=False,
                value=0,
            ),
            html.Br(),
            html.Label('scaling'),
            dcc.Slider(
                id='image-scaling-slider',
                min=-10,
                max=10,
                step=1,
                marks={
                        -8:  '.33',
                        -6:  '.4',
                        -4: '.5',
                        -2: '.67',
                         0: '1',
                         2: '1.5',
                         4: '2',
                         6: '2.5',
                         8: '3',
                        },
                included=False,
                value=0,
            ),            
            html.Br(),
            html.Label('colors'),

                    html.Label('R',style={'fontSize': 10,'font-weight':'italic'}),
                    dcc.Slider(
                        id='image-r-slider',
                        min=-1,
                        max=1,
                        step=.05,
                        marks={
                                -1:'-1',
                                -.8:'-.8',
                                -.6:'-.6',
                                -.4:'-.4',
                                 -.2:'-.2',
                                 0:'0',
                                 .2:'.2',
                                 .4:'.4',
                                 .6:'.6',
                                 .8:'.8',
                                  1:'1',
                                },
                        included=False,
                        value=0,
                    ),
 

                    html.Label('G',style={'fontSize': 10,'font-weight':'italic'}),
                    dcc.Slider(
                        id='image-g-slider',
                        min=-1,
                        max=1,
                        step=.05,
                        marks={
                                -1:'-1',
                                -.8:'-.8',
                                -.6:'-.6',
                                -.4:'-.4',
                                 -.2:'-.2',
                                 0:'0',
                                 .2:'.2',
                                 .4:'.4',
                                 .6:'.6',
                                 .8:'.8',
                                  1:'1',
                                },
                        included=False,
                        value=0,
                    ),
    

                    html.Label('B',style={'fontSize': 10,'font-weight':'italic'}),
                    dcc.Slider(
                        id='image-b-slider',
                        min=-1,
                        max=1,
                        step=.05,
                        marks={
                                -1:'-1',
                                -.8:'-.8',
                                -.6:'-.6',
                                -.4:'-.4',
                                 -.2:'-.2',
                                 0:'0',
                                 .2:'.2',
                                 .4:'.4',
                                 .6:'.6',
                                 .8:'.8',
                                  1:'1',
                                },
                        included=False,
                        value=0,
                    )
           
            ], className = "three columns"),
                
                
            html.Div([
            html.Label('Feature Visualizations', style={'fontSize': 18,'font-weight':'bold'}),
            html.Br(),
            html.Div( style=dict(display='flex'),
                children = [     
                    daq.ToggleSwitch(
                        id='featviz-nodeedge-toggle',
                        label=['node','edge    '],
                        style={'float': 'right','margin': 'auto'}
                        #labelPosition='bottom'
                    ), 
                    html.Label(''),
                    daq.ToggleSwitch(
                        id='featviz-channelneuron-toggle',
                        label=['channel','neuron    '],
                        style={'float': 'right','margin': 'auto'}
                        #labelPosition='bottom'
                    ),
                    html.Label(''),
                    daq.ToggleSwitch(
                        id='featviz-positivenegative-toggle',
                        label=['positive','negative    '],
                        style={'float': 'right','margin': 'auto'}
                        #labelPosition='bottom'
                    )
                ]),
            html.Br(),
            dcc.Graph(
               id='featviz-image',
               style={
              'width': '14vw',
              'height':'14vw'
               },
               figure=figure_init,
               config={
                       'displayModeBar': False
                       }
            ),
            html.Button('Generate', id='featviz-button')
            #html.Button('Generate', id='gen-featviz-button')
            ], className= "five columns"),
        
        
        
            html.Div([
            html.Label('Model Ablations', style={'fontSize': 18,'font-weight':'bold'}),
            dcc.Textarea(
                id='ablations-textarea',
                value='',
                style={'width': '70%', 'height': 300}),
            html.Button('Ablate', id='ablate-model-button')
            ], className= "four columns"),
        
        ], className="row"
        ),
                
#         html.Div([
#             html.Div([
#                 dcc.Markdown("""
#                     **Hover Data**

#                     Mouse over values in the graph.
#                 """),
#                 html.Pre(id='hover-data', style=styles['pre'])
#             ], className='two columns'),

#             html.Div([
#                 dcc.Markdown("""
#                     **Click Data**

#                     Click on points in the graph.
#                 """),
#                 html.Pre(id='click-data', style=styles['pre']),
#             ], className='two columns'),

#             html.Div([
#                 dcc.Markdown("""
#                     **Selection Data**

#                     Choose the lasso or rectangle tool in the graph's menu
#                     bar and then select points in the graph.

#                     Note that if `layout.clickmode = 'event+select'`, selection data also 
#                     accumulates (or un-accumulates) selected data if you hold down the shift
#                     button while clicking.
#                 """),
#                 html.Pre(id='selected-data', style=styles['pre']),
#             ], className='two columns'),

# #                 html.Div([
# #                     dcc.Markdown("""
# #                         **Zoom and Relayout Data**

# #                         Click and drag on the graph to zoom or click on the zoom
# #                         buttons in the graph's menu bar.
# #                         Clicking on legend items will also fire
# #                         this event.
# #                     """),
# #                     html.Pre(id='relayout-data', style=styles['pre']),
# #                 ], className='two columns')
                
#             html.Div([
#                 dcc.Markdown("""
#                     **Figure Data**

#                     Figure json info.
#                 """),
#                 html.Pre(id='figure-data', style=styles['pre']),
#             ], className='four columns')

#         ], className= 'row'
#         ),

    #hidden divs for storing intermediate values     
    # The memory store reverts to the default on every page refresh
    dcc.Store(id='memory',data=state),
    # The local store will take the initial data
    # only the first time the page is loaded
    # and keep it until it is cleared.
    #dcc.Store(id='local', storage_type='local'),
    # Same as the local store but will lose the data
    # when the browser/tab closes.
    #dcc.Store(id='session', storage_type='session',data=state),
    

    # hidden signal value
    html.Div(id='input-image-signal',  style={'display': 'none'}),
    html.Div(id='target-signal', style={'display': 'none'},children = [state['target_category'],state['target_node']]),
    html.Div(id='ablations-signal',  style={'display': 'none'}, children = [])
])



# perform expensive computations in this "global store"
# these computations are cached in a globally available
# 'cached' folder in the prepped_models/[model] folder
@cache.memoize()
def activations_store(image_name,ablation_list):

    print('Updating cached activations with {}'.format(image_name))
    activations = get_model_activations_from_image(get_image_path(image_name,params)[1], model_dis, params)
    
    return activations

@app.callback(Output('input-image-signal', 'children'), 
              [Input('dynamic-input-image-dropdown', 'value'),
               Input('ablations-signal', 'children')])
def update_activations_store(image_name,ablation_list):
    # compute value and send a signal when done
    activations_store(image_name,ablation_list)
    return image_name


@cache.memoize()
def ranksdf_store(target_category, target_node,ablation_list,model_dis=model_dis):
    print('Updating cached rank dfs with {}'.format(target_category))
    model_dis = clear_ranks_across_model(model_dis)
    target_type = image_category_or_contrast(target_category,params)
    target_category_nodes_df = None
    target_category_edges_df = None
    if target_type == 'category' and target_node == 'loss' and ablation_list == []:
        #edges
        if categories_edges_df is not None:
            if len(categories_edges_df.loc[categories_edges_df['category']==target_category]) > 0:
                target_category_edges_df = categories_edges_df.loc[categories_edges_df['category']==target_category]
        if target_category_edges_df is None:
            target_category_edges_df = rank_file_2_df(os.path.join(params['ranks_data_path'],'categories_edges','%s_edges_rank.pt'%target_category))   
        #node
        if categories_nodes_df is not None:
            if len(categories_nodes_df.loc[categories_nodes_df['category']==target_category]) > 0:
                target_category_nodes_df = categories_nodes_df.loc[categories_nodes_df['category']==target_category]
        if target_category_nodes_df is None:
            target_category_nodes_df = rank_file_2_df(os.path.join(params['ranks_data_path'],'categories_nodes','%s_nodes_rank.pt'%target_category))
    elif target_type == 'category':
        target_category_nodes_df,target_category_edges_df = rank_dict_2_df(get_model_ranks_for_category(target_category, target_node, model_dis,params))
    elif target_type == 'input_image':
        target_category_nodes_df,target_category_edges_df = rank_dict_2_df(get_model_ranks_from_image(get_image_path(target_category,params)[1],target_node, model_dis, params))

    else:  #contrast
        target_category_nodes_df,target_category_edges_df = contrast_str_2_dfs(target_category,target_node,model_dis,params,ablation_list)
    #print('FROM RANKS DF STORE')
    print(target_category_edges_df)
    return target_category_nodes_df,target_category_edges_df

@app.callback(Output('target-signal', 'children'), 
              [Input('input-category', 'value'),
               Input('target-node','value'),
               Input('ablations-signal', 'children')])
def update_ranksdf_store(target_category,target_node,ablation_list):
    # compute value and send a signal when done
    print('update ranksdf_store triggered')
    ranksdf_store(target_category,target_node,ablation_list)
    return [target_category,target_node]



####Call Back Functions

#Ablations
@app.callback(Output('ablations-signal', 'children'), 
              [Input('ablate-model-button', 'n_clicks')],
              [State('ablations-textarea','value')])
def update_ablations(n_clicks,text,model_dis=model_dis):
    # compute value and send a signal when done
    ablation_list = ablation_text_2_list(text, params)
    ablate_model_with_list(ablation_list,model_dis,params)
    return ablation_list


#Hidden State
@app.callback(
    Output('memory', 'data'),
    [Input('target-signal', 'children'),
     Input('node-actmap-dropdown', 'value'),
     Input('edge-actmaps-input', 'value'),
     Input('edge-thresh-slider','value'),
     Input('node-thresh-slider','value'),
     Input('layer-projection','value'),
     Input('subgraph-criterion','value')],
    [State('memory', 'data'),
     State('ablations-signal', 'children')])
def update_store(target,node_value,edge_value,edge_threshold,node_threshold,projection,rank_type,state,ablation_list):
    print('CALLED: update_store\n')
    ctx = dash.callback_context
    if not ctx.triggered:
        raise Exception('no figure updates yet')
    else:
        trigger = ctx.triggered[0]['prop_id']
    state['last_trigger'] = trigger  #store the last trigger of state change in state
    print('TRIGGER %s'%trigger)
    
    hierarchical = False
    if rank_type == 'hierarchical':
        hierarchical = True
        rank_type = 'actxgrad'
        
    target_category,target_node = target[0],target[1]
    #fetch select edges DF
    if trigger in ['target-signal.children','edge-thresh-slider.value','node-thresh-slider.value','layer-projection.value','subgraph-criterion.value']:
        if rank_type == 'weight':
            target_edges_df = weight_edges_df
            target_nodes_df = weight_nodes_df
            weight=True
        else:   
            target_nodes_df,target_edges_df = ranksdf_store(target_category,target_node,ablation_list)
            weight=False   
        target_edges_df = minmax_normalize_ranks_df(target_edges_df,params,weight=weight)
        target_nodes_df = minmax_normalize_ranks_df(target_nodes_df,params,weight=weight)

        if hierarchical:
            #nodes_thresholded_df = get_thresholded_ranksdf(node_threshold,rank_type, target_nodes_df)
            #filter_edges_df = filter_edges_by_nodes(target_edges_df,nodes_thresholded_df)
            #edges_thresholded_df = get_thresholded_ranksdf(edge_threshold,rank_type,filter_edges_df)
            #edges_thresholded_df = hierarchically_threshold_edges(edge_threshold,rank_type,target_edges_df,nodes_thresholded_df)
            print('finding hierarchical subgraph')
            start = time.time()
            nodes_thresholded_df,edges_thresholded_df = hierarchical_accum_threshold(node_threshold[0],edge_threshold[0],rank_type,target_edges_df,target_nodes_df,ascending=False)
            print('time: %s'%str(time.time() - start))
            print('found %s nodes and %s edges'%(str(len(nodes_thresholded_df)),str(len(edges_thresholded_df))))
            #node_minmax = node_threshold
            node_min = {}
            for layer in target_nodes_df['layer'].unique():
                if len(nodes_thresholded_df.loc[nodes_thresholded_df['layer']==layer]) > 1:
                    node_min[layer] = nodes_thresholded_df.loc[nodes_thresholded_df['layer']==layer][rank_type+'_rank'].min()
                else:
                    node_min[layer] = None

        else: 
            nodes_thresholded_df = None
            edges_thresholded_df = get_thresholded_ranksdf(edge_threshold,rank_type,target_edges_df)
            node_min = None

    if trigger == 'target-signal.children':
        print('changing target category to %s'%target_category)
        #print(target_nodes_df)
        state['node_colors'], state['node_weights'] = gen_node_colors(target_nodes_df,rank_type,params,node_min=node_min)
        #state['max_edge_weight'] = get_max_edge_weight(target_category)
        state['edge_positions'], state['edge_colors'], state['edge_widths'],state['edge_weights'], state['edge_names'], state['max_edge_width_indices'] = gen_edge_graphdata(edges_thresholded_df, state['node_positions'], rank_type, target_category,params)

    elif trigger == 'node-actmap-dropdown.value' or trigger == 'edge-actmaps-input.value':
        state['last_trigger'] = 'selection_change'
        print(edge_value)
        #update node if button value different than store value
        if state['node_select_history'][-1] != node_value:
            print('changing selected node to %s'%node_value)
            state['node_select_history'].append(node_value)
            if len(state['node_select_history']) > 10:
                del state['node_select_history'][0] 
        #update edge if button value different than store value
        if state['edge_select_history'][-1] != edge_value and check_edge_validity(edge_value.strip(),params)[0]:
            print('changing selected edge to %s'%edge_value)
            state['edge_select_history'].append(edge_value)
            print(state['edge_select_history'])
            if len(state['edge_select_history']) > 10:
                del state['edge_select_history'][0]              

    elif trigger == 'edge-thresh-slider.value':
        print('changing edge thresholds to %s - %s'%(edge_threshold[0],edge_threshold[1]))
        state['edge_threshold'] == edge_threshold
        print('found %s edges'%len(edges_thresholded_df))
        state['edge_positions'], state['edge_colors'], state['edge_widths'], state['edge_weights'], state['edge_names'], state['max_edge_width_indices'] = gen_edge_graphdata(edges_thresholded_df, state['node_positions'], rank_type, target_category,params)
    
    elif trigger == 'node-thresh-slider.value':
        print('changing node thresholds to %s - %s'%(node_threshold[0],node_threshold[1]))
        state['node_threshold'] == node_threshold
        print('found %s nodes'%len(nodes_thresholded_df))
        state['edge_positions'], state['edge_colors'], state['edge_widths'], state['edge_weights'], state['edge_names'], state['max_edge_width_indices'] = gen_edge_graphdata(edges_thresholded_df, state['node_positions'], rank_type, target_category,params)
        state['node_colors'], state['node_weights'] = gen_node_colors(target_nodes_df,rank_type,params,node_min=node_min)
        
        
    elif trigger == 'layer-projection.value':
        print('changing layer projection to %s\n'%projection)
        state['projection']=projection
        if projection == 'Grid':
            node_positions = all_node_positions[projection]
        else:
            node_positions = all_node_positions[projection][rank_type]
        state['edge_positions'], state['edge_colors'], state['edge_widths'],state['edge_weights'], state['edge_names'], state['max_edge_width_indices'] = gen_edge_graphdata(edges_thresholded_df, state['node_positions'], rank_type, target_category,params)

    elif trigger == 'subgraph-criterion.value':
        print('changing weighting criterion to %s\n'%rank_type)
        state['rank_type']=rank_type
        state['node_colors'], state['node_weights'] = gen_node_colors(target_nodes_df,rank_type,params,node_min=node_min)
        #state['node_positions']=format_node_positions(projection=projection,rank_type=rank_type)
        state['edge_positions'], state['edge_colors'], state['edge_widths'],state['edge_weights'], state['edge_names'], state['max_edge_width_indices'] = gen_edge_graphdata(edges_thresholded_df, state['node_positions'], rank_type, target_category,params)

    else:
        raise Exception('unknown trigger: %s'%trigger)    
    return state


#Network Graph Figure
@app.callback(
    Output('network-graph', 'figure'),
    [Input('memory', 'data')],
    [State('network-graph','figure')])
def update_figure(state, fig):
    #network_graph_layout['uirevision'] = True
    print('CALLED: update_figure\n')
    print(state['edge_threshold'])
    print(state['edge_select_history'])
    print(state['node_select_history'])
    if state['last_trigger'] == 'selection_change':   #minimal updates
        #hightlight edge
        print('updating edge highlight to %s'%state['edge_select_history'][-1])
        #if len(state['edge_select_history']) >1:
        #if state['edge_select_history'][-1] != state['edge_select_history'][-2]:  #didnt click same point
        flat_edge_names = [item for sublist in state['edge_names'] for item in sublist]
        flat_edge_colors = [item for sublist in state['edge_colors'] for item in sublist]
        try:  #update current edge if it exists to black
            #print(flat_edge_names)
            fig['data'][flat_edge_names.index(state['edge_select_history'][-1])+params['num_layers']+1]['line']['color'] = 'rgba(0,0,0,1)'
        except:
            print('select edge, %s,  not recolored as no longer shown'%state['edge_select_history'][-1])
        if len(state['edge_select_history']) > 1: #there is a previous edge to unselect
            try: #recolor previous edge if it exists from black
                fig['data'][flat_edge_names.index(state['edge_select_history'][-2])+params['num_layers']+1]['line']['color'] = flat_edge_colors[flat_edge_names.index(state['edge_select_history'][-2])]
            except:
                print('previous edge, %s,  not recolored as no longer shown'%state['edge_select_history'][-2])
        #highlight node
        print('updating node highlight to %s'%state['node_select_history'][-1])
        #if len(state['node_select_history']) >1:
        #    if state['node_select_history'][-1] != state['node_select_history'][-2]: 
                #update current node color to black
        if str(state['node_select_history'][-1]).isnumeric():  #if normal node
            select_layer,select_position,select_layer_name = nodeid_2_perlayerid(state['node_select_history'][-1],params)
            fig['data'][select_layer+1]['marker']['color'][select_position] = 'rgba(0,0,0,1)'
        else:   #imgnode
            fig['data'][0]['marker']['color'][fig['data'][0]['text'].index(state['node_select_history'][-1])] = 'rgba(0,0,0,1)'
        #update previous node color to its usual color
        if len(state['node_select_history']) > 1: #there is a previous node to unselect
            if str(state['node_select_history'][-2]).isnumeric():  #if normal node
                prev_select_layer,prev_select_position,prev_select_layer_name = nodeid_2_perlayerid(state['node_select_history'][-2],params)
                print(prev_select_layer,prev_select_position,prev_select_layer_name)
                fig['data'][prev_select_layer+1]['marker']['color'][prev_select_position] = state['node_colors'][prev_select_layer][prev_select_position]
            else:   #imgnode
                fig['data'][0]['marker']['color'][fig['data'][0]['text'].index(state['node_select_history'][-2])] = state['imgnode_colors'][fig['data'][0]['text'].index(state['node_select_history'][-2])]
        #fig['layout']['uirevision']=True   
        return fig    
    else:   #regenerate full traces
        combined_traces = gen_networkgraph_traces(state,params)
        fig['data'] = combined_traces
        #layout = network_graph_layout
        #layout['uirevision'] = True
        return fig

#Node Actmap Dropdown
@app.callback(
    Output('node-actmap-dropdown', 'value'),
    [Input('network-graph', 'clickData')],
    [State('node-actmap-dropdown', 'value')])
def switch_node_actmap_click(clickData,current_value):
    print('CALLED: switch_node_actmap_click')
    if clickData is None:
        return current_value 
        #raise Exception('no click data')
    if int(clickData['points'][0]['curveNumber']) > params['num_layers']:
        return current_value
        #raise Exception('edge was clicked')
    return clickData['points'][0]['text']

#Edge Actmaps Input
@app.callback(
    Output('edge-actmaps-input', 'value'),
    [Input('network-graph', 'clickData')],
    [State('edge-actmaps-input', 'value'),
     State('memory', 'data')])
def switch_edge_actmaps_click(clickData,current_value,state):
    print('CALLED: switch_edge_actmaps_click')
    if clickData is None:
        return current_value
        #raise Exception('no click data')
    if int(clickData['points'][0]['curveNumber']) <= params['num_layers']:
        return current_value
        #raise Exception('node was clicked')
    return get_nth_element_from_nested_list(state['edge_names'],int(clickData['points'][0]['curveNumber'])-(params['num_layers']+1))


#Node actmap graph
@app.callback(
    Output('node-actmap-graph', 'figure'),
    [Input('node-actmap-dropdown', 'value'),
     Input('relu-checkbox','value'),
     Input('input-image-signal', 'children')],
    [State('ablations-signal', 'children')])
def update_node_actmap(nodeid,relu_checked,image_name,ablation_list):       #EDIT: needs support for black and white images
    print('CALLED: update_node_actmap')
    layer, within_id,layer_name = nodeid_2_perlayerid(nodeid,params)
    #fetch activations
    if image_name in all_activations['nodes'] and ablation_list == []:
        activations = all_activations
    else:
        activations  = activations_store(image_name,ablation_list)
        
    if layer == 'img': #code for returning color channel as activation map
        #np_chan_im = get_channelwise_image(image_name,state['imgnode_names'].index(nodeid),params['input_image_directory']=params['input_image_directory'])
        np_chan_im = activations['edges_in'][image_name][0][within_id]
        return go.Figure(data=go.Heatmap( z = np.flip(np_chan_im,0), name = nodeid),
                        layout=node_actmap_layout) 
    act_map = activations['nodes'][image_name][layer][within_id]
    if relu_checked != []:
        act_map = relu(act_map)
    return go.Figure(data=go.Heatmap( z = np.flip(act_map,0),
                                      #zmin=-11,
                                      #zmax=14,
                                      colorbar = dict(thicknessmode = "fraction",thickness=.1)
                                    ),
                     layout=node_actmap_layout) 

@app.callback(
    Output('node-sum', 'children'),
    [Input('node-actmap-graph', 'figure')])
def update_node_sum(fig):
    mean = np.mean(fig['data'][0]['z'])
    return 'mean: %s'%str(mean)


#Node deepviz graph
@app.callback(
    Output('node-deepviz-image', 'figure'),
    [Input('node-actmap-dropdown', 'value')])
def update_node_deepviz(nodeid):       #EDIT: needs support for black and white images
    print('CALLED: update_node_deepviz')
    layer,within_layer_id,layer_name = nodeid_2_perlayerid(nodeid,params)    
    if layer == 'img': 
        return figure_init
    image_name = fetch_deepviz_img(model,str(nodeid),params)
    image_path = params['prepped_model_path']+'/visualizations/images/'+image_name
    return image2plot(image_path,input_image_layout)
    

#Edge deepviz graph
@app.callback(
    Output('edge-deepviz-image', 'figure'),
    [Input('edge-actmaps-input', 'value')])
def update_edge_deepviz(edgename):       #EDIT: needs support for black and white images
    print('CALLED: update_edge_deepviz')
    #layer,within_layer_id,layer_name = nodeid_2_perlayerid(nodeid,params)    
    #if layer == 'img': 
    #    return figure_init
    image_name = fetch_deepviz_img(model_dis,edgename,params)
    image_path = params['prepped_model_path']+'/visualizations/images/'+image_name
    return image2plot(image_path,input_image_layout)
     

#Node inputs actmap graph
@app.callback(
    Output('node-inputs-graph', 'figure'),
    [Input('node-actmap-dropdown', 'value'),
     Input('input-image-signal', 'children'),
     Input('target-signal', 'children'),
     Input('subgraph-criterion','value')],
     [State('ablations-signal', 'children')])
def update_node_inputs(nodeid,image_name,target,rank_type,ablation_list,model=model_dis,max_num = params['max_node_inputs']):       
    print('CALLED: update_node_inputs')
    
    hierarchical = False
    if rank_type == 'hierarchical':
        hierarchical = True
        rank_type = 'actxgrad'
    
    
    target_category,target_node = target[0],target[1]
    node_layer,node_within_layer_id,layer_name = nodeid_2_perlayerid(nodeid,params)
    #fetch activations
    if image_name in all_activations['nodes'] and ablation_list == []:
        activations = all_activations
    else:
        activations = activations_store(image_name,ablation_list)
    #fetch edges df
    if rank_type == 'weight':
        target_edges_df = weight_edges_df
    else:
        target_edges_df = ranksdf_store(target_category,target_node,ablation_list)[1]
    #return no input if on input image node 
    if node_layer == 'img':
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=[],
            y=[]))
        fig.update_layout(xaxis=dict(visible=False),
                          yaxis=dict(visible=False),
                          annotations = [dict(text="No Inputs",
                                              xref="paper",
                                              yref="paper",
                                              showarrow=False,
                                              font=dict(size=28))]
                         )
        return fig

    all_node_edges_df = target_edges_df.loc[(target_edges_df['layer']==node_layer) & (target_edges_df['out_channel'] == node_within_layer_id)]
    #if sort_images:                      
    all_node_edges_df = all_node_edges_df.sort_values(by=[rank_type+'_rank'],ascending=False)
    top_node_edges_df = all_node_edges_df.head(max_num)
    fig = make_subplots(rows=len(top_node_edges_df)+1, cols=3)
    #print(top_node_edges_df)
    i=1
    for row in top_node_edges_df.itertuples():
        if node_layer == 0:
            edge_name = str(params['imgnode_names'][row.in_channel])+'-'+str(nodeid)
        else:
            edge_name = str(params['layer_nodes'][node_layer-1][1][row.in_channel])+'-'+str(nodeid)
        #add activation map
        fig.add_trace(
               go.Heatmap(z = edgename_2_edge_figures(edge_name, image_name, kernels, activations,params)[2],
                          #zmin = -1,
                          #zmax = 1,
                          name = edge_name,
                          coloraxis="coloraxis"
                          #showscale = False,
                          #colorbar = dict(lenmode='fraction',len=1/len(top_node_edges_df), 
                          #                y=(i)/len(top_node_edges_df)-.01,
                          #                thicknessmode = "fraction",thickness=.1,
                          #                ypad=1
                          #               )
                          ),
               row=i, col=2),
        #add kernel
        fig.add_trace(
               go.Heatmap(z = edgename_2_edge_figures(edge_name, image_name, kernels, activations,params)[0],
                          #zmin = -1,
                          #zmax = 1,
                          name = edge_name+'_kernel',
                          coloraxis="coloraxis2"
                          #showscale = False,
                          #colorbar = dict(lenmode='fraction',len=1/len(top_node_edges_df), 
                          #                y=(i)/len(top_node_edges_df)-.01,
                          #                thicknessmode = "fraction",thickness=.1,
                          #                ypad=1
                          #               )
                          ),
               row=i, col=3),

        #add visualization
        viz_img_name = fetch_deepviz_img_for_node_inputs(model,edge_name,params)
        viz_img_path = params['prepped_model_path']+'/visualizations/images/'+viz_img_name
        viz_img = Image.open(viz_img_path)
        #fig.add_trace(go.Image(z=viz_img,name=viz_img_name), row=i, col=1)
        fig.add_trace(go.Scatter(x=[],y=[]),row=i,col=1)
        fig.add_layout_image(
                            source=viz_img,
                            xref="x",
                            yref="y",
                            x=0,
                            y=10,
                            sizex=10,
                            sizey=10,
                            sizing="stretch",
                            opacity=1,
                            layer="below",
                            row=i, col=1
                            )
        fig.update_xaxes(visible=False,range=(0,10),showline=False,showgrid=False,showticklabels=False,row=i,col=1)
        fig.update_yaxes(visible=False,range=(0,10),showline=False,showgrid=False,showticklabels=False,row=i,col=1)
   


        i+=1
    
    fig.update_layout(height=200*len(top_node_edges_df), 
                      width=340,
                      #yaxis=dict(scaleanchor="x", scaleratio=1/len(top_node_edges_df)),
                      #title_text="Inputs to Node",
                      #xaxis=dict(visible=False),
                      #yaxis=dict(visible=False),
                      coloraxis_showscale=False,
                      coloraxis2 = dict(showscale=False,
                                        colorscale='inferno',
                                        colorbar = dict(
                                                        thicknessmode = "fraction",thickness=.05, 
                                                        lenmode='fraction',len=.7)),
                      margin=dict(
                                    l=0,
                                    r=0,
                                    b=0,
                                    t=0,
                                    pad=0)
                     )
    fig.update_coloraxes(colorscale='inferno',colorbar = dict(
                                                              thicknessmode = "fraction",thickness=.05, 
                                                              lenmode='fraction',len=.7)
                        )
#     fig.update_coloraxes2(colorscale='inferno',colorbar = dict(
#                                                               thicknessmode = "fraction",thickness=.05, 
#                                                               lenmode='fraction',len=.7)
#                        )
    return fig







#image graph
@app.callback(
    Output('img-actmap-graph', 'figure'),
    [Input('dynamic-input-image-dropdown', 'value'),
     Input('node-actmap-graph','clickData'),
     Input('node-actmap-graph','figure')],
    [State('img-actmap-graph', 'figure'),
     State('node-actmap-dropdown', 'value')])
def update_inputimg_actmap(image_name,click_data,node_actmap_fig,image_fig,nodeid): 
    print('CALLED: update_inputimg_actmap')
    #if os.path.exists(params['input_image_directory']+image_name):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise Exception('no figure updates yet')
    else:
        trigger = ctx.triggered[0]['prop_id']
    if trigger == 'dynamic-input-image-dropdown.value':
        return image2plot(get_image_path(image_name,params)[1],input_image_layout)
    elif receptive_fields is None:
        return image2plot(get_image_path(image_name,params)[1],input_image_layout)
    elif click_data is None:
        return image2plot(get_image_path(image_name,params)[1],input_image_layout)
    else:
        #nodeid = node_actmap_fig['data'][0]['name']
        layer_name = nodeid_2_perlayerid(nodeid,params)[2]
        if layer_name == 'img':
            raise Exception('no receptive fields for input image actmap')
        heatmap_dim_y = len(node_actmap_fig['data'][0]['z'])
        heatmap_dim_x = len(node_actmap_fig['data'][0]['z'][0]) 
        x_click = click_data['points'][0]['x']
        y_click = heatmap_dim_y - click_data['points'][0]['y']-1
        print('x_click')
        print(x_click)
        print('y_click')
        print(y_click)
        recep_field = receptive_field_for_unit(receptive_fields, layer_name, (x_click,y_click))
        recep_field_normed = [[recep_field[0][0]*10/input_image_size,recep_field[0][1]*10/input_image_size],
                              [recep_field[1][0]*10/input_image_size,recep_field[1][1]*10/input_image_size]]
        print('normalized')
        print(recep_field_normed)
        x_points = [recep_field_normed[0][0],recep_field_normed[0][0],recep_field_normed[0][1],recep_field_normed[0][1],recep_field_normed[0][0]]
        y_points = [10 - recep_field_normed[1][0],10 - recep_field_normed[1][1],10 - recep_field_normed[1][1],10 - recep_field_normed[1][0],10 - recep_field_normed[1][0]]
        print('x points')
        print(x_points)
        print('y points')
        print(y_points)
        image_fig['data'] = [{'mode': 'lines', 'x': x_points, 'y': y_points, 'type': 'scatter'}]
        return image_fig
        



    #else:
        #return image2plot(params['prepped_model_path']+'/visualizations/'+image_name,input_image_layout)
    
# #image dropdown
# @app.callback(
#     Output('input-image-dropdown', 'options'),
#     [Input('node-deepviz-image', 'figure'),
#      Input('edge-deepviz-image', 'figure')])
# def update_inputimg_dropdown(node_fig,edge_fig): 
#     print('CALLED: update_inputimg_dropdown options')
#     return [{'label': i, 'value': i} for i in params['input_image_list']+os.listdir(params['prepped_model_path']+'/visualizations/images/')]

#dynamic dropdown
@app.callback(
    dash.dependencies.Output("dynamic-input-image-dropdown", "options"),
    [dash.dependencies.Input("dynamic-input-image-dropdown", "search_value")],
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [{'label': i, 'value': i} for i in params['input_image_list']+os.listdir(params['prepped_model_path']+'/visualizations/images/') if search_value in i]





#kernel
@app.callback(
    Output('edge-kernel-graph', 'figure'),
    [Input('edge-actmaps-input','value')],
    [State('edge-kernel-graph','figure')])
def update_edge_kernelmap(edge_name,figure):
    print('CALLED: update_edge_kernelmap')
    kernel,inmap,outmap = edgename_2_edge_figures(edge_name, None, kernels, None,params)
    if kernel is not None:
        return go.Figure(data=go.Heatmap(z = kernel,
                                         #zmin=-.5,
                                         #zmax=.5,
                                         colorbar = dict(thicknessmode = "fraction",thickness=.1)),
                         layout=kernel_layout)
    else:
        return figure
                

#edge in        
@app.callback(
    Output('edge-inmap-graph', 'figure'),
    [Input('edge-actmaps-input','value'),
     Input('input-image-signal', 'children')],
    [State('edge-inmap-graph','figure'),
     State('ablations-signal', 'children')])
def update_edge_inmap(edge_name,image_name,figure,ablation_list):
    print('CALLED: update_edge_inmap')
    #fetch activations
    if image_name in all_activations['nodes'] and ablation_list == []:
        activations = all_activations
    else:
        activations = activations_store(image_name, ablation_list)
        
    kernel,inmap,outmap = edgename_2_edge_figures(edge_name, image_name, kernels, activations,params)
    if inmap is not None:
        return go.Figure(data=go.Heatmap(z = inmap,
                                         #zmin=-2,zmax=2,
                                         colorbar = dict(thicknessmode = "fraction",thickness=.1)
                                        ),
                         layout=edge_inmap_layout)
    else:
        print('edge inmap error')
        return figure

@app.callback(
    Output('edgein-sum', 'children'),
    [Input('edge-inmap-graph', 'figure')])
def update_node_sum(fig):
    mean = np.mean(fig['data'][0]['z'])
    return 'mean: %s'%str(mean)    

#edge out
@app.callback(
    Output('edge-outmap-graph', 'figure'),
    [Input('edge-actmaps-input','value'),
     Input('input-image-signal', 'children')],
    [State('edge-outmap-graph','figure'),
     State('ablations-signal', 'children')])
def update_edge_outmap(edge_name,image_name,figure, ablation_list):
    print('CALLED: update_edge_outmap')
    #fetch activations
    if image_name in all_activations['nodes'] and ablation_list == []:
        activations = all_activations
    else:
        activations = activations_store(image_name,ablation_list)
        
    kernel,inmap,outmap = edgename_2_edge_figures(edge_name, image_name, kernels, activations,params)
    if outmap is not None:
        return go.Figure(data=go.Heatmap(z = outmap,
                                         #zmin=-11,
                                         #zmax=14,
                                         colorbar = dict(thicknessmode = "fraction",thickness=.1)
                                        ),
                         layout=edge_outmap_layout)
    else:
        print('edge outmap error')
        return figure
        
@app.callback(
    Output('edgeout-sum', 'children'),
    [Input('edge-outmap-graph', 'figure')])
def update_node_sum(fig):
    mean = np.mean(fig['data'][0]['z'])
    return 'mean: %s'%str(mean)


 
#feature viz graph
@app.callback(
    Output('featviz-image', 'figure'),
    [Input('featviz-button','n_clicks')],
    [State('featviz-nodeedge-toggle', 'value'),
     State('featviz-channelneuron-toggle', 'value'),
     State('node-actmap-dropdown','value'),
     State('edge-actmaps-input','value')],
    prevent_initial_call=True)
def update_featviz_image(n_clicks,edge,neuron,nodeid,edgeid):       #EDIT: needs support for black and white images
    print('CALLED: update_featviz')
    if edge:
        image_name = regen_visualization(model_dis,edgeid,neuron,params)
    else:
        layer,within_layer_id,layer_name = nodeid_2_perlayerid(nodeid,params)    
        if layer == 'img': 
            return figure_init
        image_name = regen_visualization(model_dis,nodeid,neuron,params)
        
    image_path = params['prepped_model_path']+'/visualizations/images/'+image_name
    return image2plot(image_path,input_image_layout)


# #JSON INFO

# @app.callback(
#     Output('hover-data', 'children'),
#     [Input('node-actmap-graph', 'hoverData')])
# def display_hover_data(hoverData):
#     return json.dumps(hoverData, indent=2)




# @app.callback(
#     Output('click-data', 'children'),
#     [Input('network-graph', 'clickData')])
# def display_click_data(clickData):
#     return json.dumps(clickData, indent=2)


# @app.callback(
#     Output('selected-data', 'children'),
#     [Input('network-graph', 'selectedData')])
# def display_selected_data(selectedData):
#     return json.dumps(selectedData, indent=2)


# @app.callback(
#     Output('figure-data', 'children'),
#     [Input('input-category', 'value'),
#      Input('network-graph', 'clickData'),
#      Input('edge-thresh-slider','value'),
#      Input('memory','data')])
# def display_trigger(target_category,clickData,edge_thresh,state):
#     ctx = dash.callback_context
#     if not ctx.triggered:
#         raise Exception('no figure updates yet')
#     else:
#         trigger = ctx.triggered[0]['prop_id'].split('.')[0]
#     ctx_msg = json.dumps({
#         'states': ctx.states,
#         'triggered': ctx.triggered,
#         'inputs': ctx.inputs,
#         'full_state':state
#     }, indent=2)
#     return ctx_msg

setting up dash app


## Subgraph Sizes

In [15]:
# #Subgraph Size DF


# model_dis.to('cuda:1')
# subgraph_sizes={}
# start = time.time()
# for node in range(256,1151):
#     n_df,e_df=ranksdf_store('small_SPAN', str(node), [],model_dis=model_dis)
#     n_df = minmax_normalize_ranks_df(n_df,params)
#     threshed_n_df,threshed_e_df = hierarchical_accum_threshold(.9,.9,rank_type,e_df,n_df)
#     subgraph_sizes[node] = (len(threshed_n_df),len(threshed_e_df))
# print(time.time() - start)

In [16]:
#pickle.dump(subgraph_sizes,open('prepped_models/alexnet_sparse/subgraphs/info/subgraph_sizes_.9.9.pkl','wb'))

In [17]:
# subgraph_sizes = pickle.load(open('prepped_models/alexnet_sparse/subgraphs/info/subgraph_sizes_.9.9.pkl','rb'))

# big_list = []
# columns = ['node','layer','node_size','edge_size']
# node_sizes = []
# edge_sizes = []
# for nodes in params['layer_nodes'][2:]:
#     layer = nodes[0]
#     for node in nodes[1]:
#         try:
#             node_size= subgraph_sizes[node][0]
#             edge_size= subgraph_sizes[node][1]
#             big_list.append([node,layer,node_size,edge_size])
#         except:
#             pass

        
# subgraph_sizes_df = pd.DataFrame(big_list,columns = columns)

In [22]:
model
nodeid_2_perlayerid(1151,params)

(4, 255, 'features_10')

In [26]:
layer_sizes = {1:3*64+64,
               2:3*64+64*192+192,
               3:3*64+64*192+192*384+384,
               4:3*64+64*192+192*384+384*256+256}


subgraph_size_ratios = {
                '.9.9':[],
                '0.80.9':[],
                '0.60.8':[],
                '0.40.7':[],
                '0.20.6':[]
            }
for size in subgraph_size_ratios:
    print(size)
    for node in range(64,1152):
        try:
            sub_dict_path = 'prepped_models/alexnet_sparse/subgraphs/models/%s_%s.pt'%(str(node),size)
            submodel_dict = torch.load(sub_dict_path)
            layer,within_layer_id,layer_name = nodeid_2_perlayerid(node,params)
            subgraph_size_ratios[size].append(len(submodel_dict['edge_df'])/layer_sizes[layer])
        except:
            pass


.9.9
0.80.9
0.60.8
0.40.7
0.20.6


In [31]:
average_size_ratios = []
for size in subgraph_size_ratios:
    average_size_ratios.append(np.mean(np.array(subgraph_size_ratios[size])))

In [32]:
average_size_ratios

[0.2110308423689826,
 0.14482404578884533,
 0.059269118604168274,
 0.024244886870054386,
 0.009123183812157348]

## generating sized subgraphs

In [18]:
def get_layer_sizes_from_df(df):
    '''takes a df with a "layer" column and returns a list of layerwise sizes '''
    layers = df['layer'].unique()
    layers.sort()
    output = []
    for layer in layers:
        output.append(len(df.loc[df['layer']==layer]))
    return output

#best_size_n_df,best_size_e_df = hierarchical_size_threshold(get_layer_sizes_from_df(sub_dict['node_df']),get_layer_sizes_from_df(sub_dict['edge_df']),rank_type,n_df,e_df,selection='worst')

In [19]:
#A function that runs

def make_same_sized_subgraph(sub_dict_path,model,selection='random',node=None,save=False):
    #import pdb;pdb.set_trace()
    sub_dict = torch.load(sub_dict_path)
    if 'input' in sub_dict['gen_params'].keys():
        inputs = sub_dict['gen_params']['input']
    else:
        inputs = 'small_SPAN'
    if 'output' in sub_dict['gen_params'].keys():
        output = int(sub_dict['gen_params']['output'])
    elif node is None:
        int(sub_dict_path.split('/')[-1].split('_')[0])
    else:
        output = int(node)
    n_df,e_df=ranksdf_store(inputs, output, [], model_dis=model_dis)
    n_df = minmax_normalize_ranks_df(n_df,params)
    n_sizes = get_layer_sizes_from_df(sub_dict['node_df'])
    e_sizes = get_layer_sizes_from_df(sub_dict['edge_df'])
    size_n_df,size_e_df = hierarchical_size_threshold(n_sizes,e_sizes,rank_type,n_df,e_df,selection=selection)
    size_model = extract_subgraph(model,size_n_df,size_e_df,params)
    save_object = {
                   'model':size_model,
                   'gen_params':
                        {'node_sizes':n_sizes,
                         'edge_sizes':e_sizes,
                         'input':inputs,
                         'output':output,
                         'selection':selection,
                         'from_model':sub_dict_path
                        },
                    'node_df':size_n_df,
                    'edge_df':size_e_df
                  }
    if save:
        print('saving model to %s'%save)
        torch.save(save_object,save)
    return save_object
        

In [20]:
#run through all subgraphs generating same size subgraphs

import time
start = time.time()

for node in range(825,1152):
    try:
        sub_dict_path = 'prepped_models/alexnet_sparse/subgraphs/models/%s_.9.9.pt'%str(node)
        for version in ['random','worst']:
            save_path = 'prepped_models/alexnet_sparse/subgraphs/models/%s_models/%s_.9.9.pt'%(version,str(node))
            make_same_sized_subgraph(sub_dict_path,model,selection=version,node=node,save=save_path)
    except:
        print('node %s failed'%str(node))

print(time.time()-start)

Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "825"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_8
185
tensor(-1.1005, device='cuda:2', grad_fn=<SelectBackward>)
target node 825 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_8
185
tensor(-1.3161, device='cuda:2', grad_fn=<SelectBackward>)
target node 825 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_8
185
tensor(-1.0966, device='cuda:2', grad_fn=<SelectBackward>)
target node 825 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_8
185
tensor(-1.7845, device='cuda:2', grad_fn=<SelectBackward>)
target node 825 reached, halted forward pass
batch 4
target reached, breaking model forward pass in features_8
185
tensor(-0.9672, device='cuda:2', grad_fn=<SelectBackward>)
target node 825 reached, halted forward pass
batch 5
t

        edge_num   layer_name  layer  out_channel  in_channel  act_rank  \
0              0   features_0      0            0           0  1.312708   
1              1   features_0      0            0           1  1.933793   
2              2   features_0      0            0           2  0.614869   
3              3   features_0      0            1           0  0.494262   
4              4   features_0      0            1           1  1.509352   
...          ...          ...    ...          ...         ...       ...   
250043    250043  features_10      4          255         251  0.000000   
250044    250044  features_10      4          255         252  0.000000   
250045    250045  features_10      4          255         253  0.000000   
250046    250046  features_10      4          255         254  0.000000   
250047    250047  features_10      4          255         255  0.000000   

        grad_rank  actxgrad_rank  
0        0.000010       0.000021  
1        0.000010       0.000

layer: 0
layer: 1
layer: 2
layer: 3
saving model to prepped_models/alexnet_sparse/subgraphs/models/random_models/829_.9.9.pt
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "829"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_8
189
tensor(-0.5551, device='cuda:2', grad_fn=<SelectBackward>)
target node 829 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_8
189
tensor(-0.7831, device='cuda:2', grad_fn=<SelectBackward>)
target node 829 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_8
189
tensor(-1.7184, device='cuda:2', grad_fn=<SelectBackward>)
target node 829 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_8
189
tensor(-1.5005, device='cuda:2', grad_fn=<SelectBackward>)
target node 829 reached, halted forward pass
batch 4
target reached, breaking model forward pass in fea

layer: 0
layer: 1
layer: 2
layer: 3
saving model to prepped_models/alexnet_sparse/subgraphs/models/random_models/831_.9.9.pt
maximum value 0 for rank type act and layer 4
maximum value 0 for rank type grad and layer 4
maximum value 0 for rank type actxgrad and layer 4
layer: 0
layer: 1
layer: 2
layer: 3
saving model to prepped_models/alexnet_sparse/subgraphs/models/worst_models/831_.9.9.pt
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "832"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_8
192
tensor(-3.1957, device='cuda:2', grad_fn=<SelectBackward>)
target node 832 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_8
192
tensor(-2.8111, device='cuda:2', grad_fn=<SelectBackward>)
target node 832 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_8
192
tensor(-2.2364, device='cuda:2', grad_fn=<SelectBackward>)
target no

target node 834 reached, halted forward pass
batch 6
target reached, breaking model forward pass in features_8
194
tensor(-1.2327, device='cuda:2', grad_fn=<SelectBackward>)
target node 834 reached, halted forward pass
batch 7
target reached, breaking model forward pass in features_8
194
tensor(0.3349, device='cuda:2', grad_fn=<SelectBackward>)
target node 834 reached, halted forward pass
        edge_num   layer_name  layer  out_channel  in_channel  act_rank  \
0              0   features_0      0            0           0  1.312708   
1              1   features_0      0            0           1  1.933793   
2              2   features_0      0            0           2  0.614869   
3              3   features_0      0            1           0  0.494262   
4              4   features_0      0            1           1  1.509352   
...          ...          ...    ...          ...         ...       ...   
250043    250043  features_10      4          255         251  0.000000   
250044  

...

layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/worst_models/1139_.9.9.pt
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "1140"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_10
244
tensor(-1.9361, device='cuda:2', grad_fn=<SelectBackward>)
target node 1140 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_10
244
tensor(-1.4075, device='cuda:2', grad_fn=<SelectBackward>)
target node 1140 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_10
244
tensor(-1.2946, device='cuda:2', grad_fn=<SelectBackward>)
target node 1140 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_10
244
tensor(-1.0182, device='cuda:2', grad_fn=<SelectBackward>)
target node 1140 reached, halted forward pass
batch 4
target reached, breaking model f

layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/random_models/1142_.9.9.pt
layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/worst_models/1142_.9.9.pt
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "1143"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_10
247
tensor(-2.9114, device='cuda:2', grad_fn=<SelectBackward>)
target node 1143 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_10
247
tensor(-2.8883, device='cuda:2', grad_fn=<SelectBackward>)
target node 1143 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_10
247
tensor(-4.0019, device='cuda:2', grad_fn=<SelectBackward>)
target node 1143 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_10
247
tensor(-3.

layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/random_models/1145_.9.9.pt
layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/worst_models/1145_.9.9.pt
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "1146"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_10
250
tensor(-1.2364, device='cuda:2', grad_fn=<SelectBackward>)
target node 1146 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_10
250
tensor(-1.3931, device='cuda:2', grad_fn=<SelectBackward>)
target node 1146 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_10
250
tensor(-0.5175, device='cuda:2', grad_fn=<SelectBackward>)
target node 1146 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_10
250
tensor(-1.

layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/random_models/1148_.9.9.pt
layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/worst_models/1148_.9.9.pt
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "1149"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_10
253
tensor(-1.5446, device='cuda:2', grad_fn=<SelectBackward>)
target node 1149 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_10
253
tensor(-1.3482, device='cuda:2', grad_fn=<SelectBackward>)
target node 1149 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_10
253
tensor(-1.4077, device='cuda:2', grad_fn=<SelectBackward>)
target node 1149 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_10
253
tensor(-1.

layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/random_models/1151_.9.9.pt
layer: 0
layer: 1
layer: 2
layer: 3
layer: 4
saving model to prepped_models/alexnet_sparse/subgraphs/models/worst_models/1151_.9.9.pt
6439.322687625885


## Model comparison

scripts for generating plots that compare the difference in neural responses to different images across models

In [15]:
def same_node_subgraph_responses(node,model,submodel, category, params,batch_size = params['batch_size']):
    '''model is model from which subgraph was extracted, terminating in "node" (use unique node id)'''
    device = params['device']
    node_layer,node_within_layer_id,node_layer_name = nodeid_2_perlayerid(node,params)
    model.to(device)
    submodel.to(device)
    
    ###Hook model
    model_activations = []
    def get_activation(node_within_layer_id=node_within_layer_id):
        def hook(model, input, output):
            model_activations.append(output[:,node_within_layer_id,:,:].detach())
            #if model_activations is None:
            #    model_activations = output[:,node,:,:].detach()
            #else:
            #    model_activations = torch.cat((model_activations,output[:,node,:,:].detach()), 0)
        return hook
    
    ##### THIS IS NOT MODEL GENERAL, ASSUMING 'FEATURES', basically alexnet only right now
    handle = model.features[int(node_layer_name.split('_')[-1])].register_forward_hook(get_activation())
    
    submodel_activations = None
    
    ####SET UP DATALOADER
    kwargs = {'num_workers': params['num_workers'], 'pin_memory': True} if ('cuda' in params['device']) else {}
    
    if category =='overall':
        categories = os.listdir(params['rank_img_path'])
    else:
        categories = [category]
    for cat in categories:

        image_loader = torch.utils.data.DataLoader(
                rank_image_data(params['rank_img_path']+'/'+cat,params['preprocess'],params['label_file_path']),
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
        ###run images through
        for i, (batch, target) in enumerate(image_loader):
            #print('batch %s'%i)
            batch, target = batch.to(device), target.to(device)
            model_output = model(batch)    #running forward pass sets up hooks and stores activations in each dissected_Conv2d module
            submodel_output = submodel(batch)
            if submodel_activations is None:
                submodel_activations = submodel_output.detach()
            else:
                submodel_activations = torch.cat((submodel_activations,submodel_output.detach()), 0)
    relu = nn.ReLU(inplace=True)          
    model_activations = relu(torch.cat(model_activations, 0))
    submodel_activations = relu(torch.squeeze(submodel_activations,1))
    #print(model_activations.shape)
    #print(submodel_activations.shape)
    handle.remove()
    return torch.flatten(model_activations),torch.flatten(submodel_activations)
            


        

In [19]:
from scipy.stats.stats import pearsonr

'''
responses = {
                '.9.9':{'best':{},'random':{},'worst':{}},
                '0.80.9':{'best':{},'random':{},'worst':{}},
                '0.60.8':{'best':{},'random':{},'worst':{}},
                '0.40.7':{'best':{},'random':{},'worst':{}},
                '0.20.6':{'best':{},'random':{},'worst':{}}
            }
'''

for size in responses:
    if size == '.9.9':
        rang = range(778,1152)
    else:
        rang = range(64,1152)
    for node in rang:
        print(node)
        for v in ['best','random','worst']:
            try:
                if v == 'best':
                    submodel_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/%s_%s.pt'%(str(node),size))
                else:
                    submodel_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/%s_models/%s_%s.pt'%(v,str(node),size))
                submodel = submodel_dict['model']
                model_act,submodel_act = same_node_subgraph_responses(node,model,submodel, 'small_SPAN', params,batch_size = 18)

                #print(v)

                diffs = torch.abs(model_act-submodel_act)
                av,sd = round(float(torch.mean(diffs)),4),round(float(torch.std(diffs)),4)

                model_act,submodel_act = model_act.cpu().numpy(),submodel_act.cpu().numpy()
                corr = pearsonr(submodel_act,model_act)[0] 

                print(str(corr)+'  ,  '+str(av)+ '  ,  '+str(sd))
                responses[size][v][node] = (corr,av,sd)
            except:
                pass


778
0.7745696660048859  ,  0.5116  ,  0.7652
0.13940580799706243  ,  0.7978  ,  1.0169
0.19354182284293464  ,  0.7515  ,  1.0987
779
0.806221110504077  ,  0.4545  ,  0.6532
0.29132016007305794  ,  0.9409  ,  0.8724
0.06219288175060243  ,  0.9074  ,  1.1058
780
0.860918179185226  ,  0.6756  ,  0.8713
0.41384233491230316  ,  0.7715  ,  0.9377
nan  ,  0.7059  ,  1.3333
781
0.7826688953639427  ,  0.4977  ,  0.6873
0.15808855253604095  ,  0.8612  ,  1.2801
-0.13888289048990415  ,  0.8888  ,  1.1705
782
0.8080083964659365  ,  0.4836  ,  0.8277
0.16659495393149842  ,  0.8738  ,  1.5586
0.04293929789584785  ,  0.8845  ,  1.5739
783
0.8342742972534697  ,  0.4751  ,  0.88
0.16307719851268304  ,  0.8639  ,  1.679
-0.0287325324723664  ,  0.9457  ,  1.5815
784
0.8183514755503213  ,  0.4999  ,  0.7163
-0.0069905626832210445  ,  1.0062  ,  1.3648
0.4012826766232229  ,  1.0404  ,  1.166
785
0.7087562345056948  ,  0.3569  ,  0.7949
0.3242167397725074  ,  0.4997  ,  1.0838
0.28418274473516425  ,  0.68  

0.2888667084003023  ,  0.978  ,  1.265
nan  ,  0.6834  ,  1.6465
846
0.8102136690513437  ,  0.3387  ,  0.6211
0.29220400488617576  ,  0.5776  ,  1.1426
-0.013164140786358164  ,  0.6358  ,  1.0945
847
0.8599663091406242  ,  0.1971  ,  0.5112
0.004599782830023859  ,  0.3995  ,  0.986
nan  ,  0.3444  ,  1.0146
848
0.8721450908408872  ,  0.2114  ,  0.608
0.24682812284095634  ,  0.3718  ,  1.2982
nan  ,  0.3713  ,  1.3116
849
0.8268125374860223  ,  0.1566  ,  0.5499
0.0694728629267908  ,  1.3811  ,  1.0566
0.03831083087786079  ,  0.2927  ,  0.9255
850
0.7581710197817373  ,  0.466  ,  0.6887
0.00715820284378593  ,  0.5799  ,  1.0905
0.001310820187532792  ,  0.5854  ,  1.0741
851
0.8759220070556203  ,  0.2469  ,  0.5951
0.039034039974787875  ,  0.512  ,  1.2352
-0.05466223269508644  ,  0.4832  ,  1.2471
852
0.7337337406100306  ,  0.3451  ,  0.8229
0.3253254416994815  ,  0.6224  ,  1.011
0.016538935878056098  ,  0.6542  ,  1.0674
853
0.80723358782689  ,  0.4933  ,  0.8507
0.30964075634681604  

nan  ,  0.3275  ,  1.0499
912
0.6620495924874641  ,  0.2983  ,  0.6322
0.13135805552089824  ,  0.4495  ,  0.786
0.015319248406018381  ,  0.7106  ,  0.6008
913
0.8053247415757177  ,  0.1591  ,  0.5573
0.04281058184005139  ,  0.2142  ,  0.8992
0.034657856558911126  ,  0.2354  ,  0.8896
914
0.7959149323935295  ,  0.1467  ,  0.4646
0.16506304960026508  ,  0.2204  ,  0.7768
nan  ,  0.2208  ,  0.779
915
0.48119249778016093  ,  0.2595  ,  0.5927
0.04078811910781729  ,  0.4274  ,  0.5951
0.04713102649257808  ,  0.3852  ,  0.6002
916
0.4840845250597694  ,  0.2972  ,  0.7096
0.06784225258659118  ,  0.2361  ,  0.7146
-0.032526296128342  ,  0.3046  ,  0.666
917
0.8239569178854556  ,  0.1746  ,  0.4712
-0.01927538085378559  ,  0.3196  ,  0.8403
-0.0359118457098837  ,  0.4852  ,  0.7044
918
0.5465600658490842  ,  0.1893  ,  0.4216
-0.009182820603163316  ,  0.9804  ,  0.3025
0.005877578251859592  ,  0.9103  ,  0.2781
919
0.6571976587380403  ,  0.2082  ,  0.5126
0.011829229580117402  ,  0.5693  ,  0.5

-0.003813834843631318  ,  0.2413  ,  0.7455
nan  ,  0.2412  ,  0.7456
977
0.7513528144640182  ,  0.237  ,  0.4846
0.05131837994105312  ,  0.5013  ,  0.6086
-0.02132398155866287  ,  0.6506  ,  0.4768
978
0.6046537731225083  ,  0.1797  ,  0.4974
0.062446708459775305  ,  0.2112  ,  0.6235
nan  ,  0.2112  ,  0.6239
979
0.7701936369531646  ,  0.2242  ,  0.4911
0.28953171131095634  ,  0.419  ,  0.6998
-0.16457680517398876  ,  0.6664  ,  0.5406
980
0.5652514013997835  ,  0.2503  ,  0.7812
0.06908586477591543  ,  0.2629  ,  0.3164
0.025862239310551243  ,  0.1659  ,  0.3513
981
0.7378551175419392  ,  0.1889  ,  0.5759
-0.024059247419170253  ,  0.2495  ,  0.8828
nan  ,  0.2243  ,  0.8933
982
0.6877247395704267  ,  0.2418  ,  0.5594
-0.05686627318477787  ,  0.4591  ,  0.6661
-0.032584093655949706  ,  0.5507  ,  0.5664
983
0.6528648326772765  ,  0.4769  ,  0.7596
0.07459320956833727  ,  0.7534  ,  0.4814
0.005416204947894434  ,  0.6555  ,  0.4918
984
0.7161872402503584  ,  0.2188  ,  0.6087
0.0435

0.0917554534459913  ,  0.6756  ,  0.4981
-0.11974828397873323  ,  0.7586  ,  0.4323
1042
0.6305261214591823  ,  0.3003  ,  0.5832
0.04322912865405094  ,  0.3795  ,  0.5608
0.020619130115812682  ,  0.4772  ,  0.4634
1043
0.6619649841296089  ,  0.1558  ,  0.4955
-0.028939919869113064  ,  0.1647  ,  0.6641
0.07695667871359957  ,  0.1717  ,  0.6549
1044
0.37027895792164167  ,  0.2442  ,  0.6871
-0.00489258338473559  ,  0.2575  ,  0.7464
-0.007406675060150737  ,  0.3553  ,  0.6666
1045
0.764218371552041  ,  0.2107  ,  0.4681
0.2766828477830463  ,  0.6236  ,  0.4888
-0.06849999821752692  ,  0.8408  ,  0.3934
1046
0.5753692934359091  ,  0.2373  ,  0.6561
-0.02390598305798699  ,  0.2518  ,  0.8408
0.07411977677949971  ,  0.3381  ,  0.7799
1047
0.7925507322600804  ,  0.2472  ,  0.6709
0.0195443589570164  ,  0.2896  ,  0.7305
-0.028821943062436753  ,  0.4511  ,  0.6275
1048
0.7948513288235486  ,  0.2968  ,  0.5605
0.2651029529659757  ,  0.4259  ,  0.6716
-0.025971410024236787  ,  0.6957  ,  0.52

...

-0.009146948719744523  ,  0.8113  ,  1.4526
654
0.1860339720727261  ,  0.8129  ,  1.3575
0.08361860803993369  ,  0.7818  ,  1.4313
0.020958070739997323  ,  0.7764  ,  1.4392
655
0.07480390968982673  ,  0.589  ,  1.3014
0.018585585352919824  ,  0.5956  ,  1.2938
-0.03688874796652585  ,  0.5857  ,  1.3006
656
0.027527665166441142  ,  0.53  ,  1.1944
0.005168816924472015  ,  0.531  ,  1.1882
0.04567731258617822  ,  0.5311  ,  1.1881
657
0.11752862952266604  ,  0.3541  ,  1.3251
nan  ,  0.3225  ,  1.3507
nan  ,  0.3225  ,  1.3507
658
0.15401789893362505  ,  0.5658  ,  1.1233
0.06706454234497827  ,  0.7824  ,  0.8895
0.21648004352477124  ,  0.7954  ,  0.8783
659
-0.015181391303149949  ,  0.5391  ,  1.4776
0.128440327234144  ,  0.5742  ,  1.4274
0.05386254687573414  ,  0.5801  ,  1.4338
660
0.13286469160485542  ,  0.3589  ,  1.0247
0.06455449695038486  ,  0.3984  ,  1.0082
0.046378923876586736  ,  0.377  ,  1.0247
661
0.06775260468650485  ,  1.0758  ,  1.101
-0.12949202283921968  ,  0.9447  

-0.12207902035929018  ,  0.7727  ,  1.3232
0.017285147627839628  ,  0.7774  ,  1.3058
724
-0.015691739344083924  ,  0.7831  ,  1.2912
0.18016928937236903  ,  0.822  ,  1.1564
0.0020449607079489063  ,  0.827  ,  1.1489
725
0.04280047900954198  ,  0.7502  ,  1.0938
0.10834340296949724  ,  0.6998  ,  1.0857
-0.09906412047849  ,  0.6997  ,  1.0877
726
nan  ,  0.4647  ,  1.5586
nan  ,  0.4647  ,  1.5586
nan  ,  0.4647  ,  1.5586
727
nan  ,  0.5337  ,  1.277
nan  ,  0.5337  ,  1.277
nan  ,  0.5337  ,  1.277
728
0.03197283737568707  ,  0.7649  ,  1.3442
0.06371159912623023  ,  0.8231  ,  1.2397
-0.026703568829901353  ,  0.8183  ,  1.2493
729
-0.0021030586837172435  ,  0.6409  ,  1.059
nan  ,  0.6408  ,  1.0592
nan  ,  0.6408  ,  1.0592
730
0.08513313076989891  ,  0.5807  ,  1.2706
-0.05581791212434881  ,  0.5069  ,  1.3485
0.023186456370465132  ,  0.5159  ,  1.3387
731
0.0970398343274335  ,  0.8491  ,  1.2772
0.043460515765933815  ,  0.8865  ,  1.2323
0.07030075459598219  ,  0.8783  ,  1.2515

0.025429193875135068  ,  0.8414  ,  1.5427
nan  ,  0.8413  ,  1.5434
809
810
nan  ,  0.5776  ,  1.1209
nan  ,  0.5776  ,  1.1209
nan  ,  0.5776  ,  1.1209
811
812
0.0651766945819487  ,  0.5066  ,  1.0163
0.04729850346412669  ,  0.5937  ,  0.8835
0.05680528559561954  ,  0.5934  ,  0.8845
813
814
nan  ,  0.9308  ,  1.6174
nan  ,  0.9308  ,  1.6174
nan  ,  0.9308  ,  1.6174
815
816
0.008920991337225577  ,  0.9189  ,  1.2167
0.008376143265834435  ,  0.6269  ,  1.422
-0.023009457751998405  ,  0.6165  ,  1.4348
817
818
0.03846016948077734  ,  0.7442  ,  0.7114
0.03168884830126236  ,  0.4854  ,  0.8947
-0.009299601222332995  ,  0.5168  ,  0.8615
819
820
0.08703085896495023  ,  0.4435  ,  1.1322
0.15802633593376786  ,  0.5975  ,  0.9761
-0.04331200960907744  ,  0.5956  ,  0.9809
821
822
nan  ,  0.7652  ,  1.4901
nan  ,  0.7652  ,  1.4901
nan  ,  0.7652  ,  1.4901
823
824
0.03455004534704808  ,  0.8082  ,  1.4011
0.04184848651962626  ,  0.8154  ,  1.3949
0.06415411736435814  ,  0.8176  ,  1.387

0.0445216447825654  ,  0.8531  ,  0.6323
0.01754920601613564  ,  0.536  ,  0.7629
0.02769279220434867  ,  0.4971  ,  0.7862
941
942
-0.0940910536524566  ,  0.3699  ,  0.9881
0.037129812707229363  ,  0.3548  ,  0.9871
nan  ,  0.3254  ,  1.012
943
944
0.03784171969342302  ,  0.6205  ,  0.5683
0.11283513668118277  ,  0.3392  ,  0.7801
0.13456101472305737  ,  0.369  ,  0.7477
945
946
0.023311401218352616  ,  0.4603  ,  0.8527
0.030087219368009968  ,  0.5366  ,  0.7781
0.007455092441155206  ,  0.4794  ,  0.8267
947
948
0.018972632077528744  ,  0.8197  ,  0.5113
-0.05160785349205383  ,  0.5821  ,  0.6591
0.20989802823722198  ,  0.5576  ,  0.6817
949
950
0.06845951920311556  ,  0.4989  ,  0.2926
-0.027625523089172644  ,  0.5245  ,  0.2728
0.02403817056925199  ,  0.5546  ,  0.263
951
952
-0.2676286313230682  ,  0.6128  ,  0.9454
-0.06050809629825232  ,  0.5342  ,  0.9825
-0.05632013073481777  ,  0.5493  ,  0.9665
953
954
0.15265658096045773  ,  0.6312  ,  1.0411
0.021318926569334012  ,  0.6897

-0.023549248408195375  ,  0.7537  ,  0.4123
0.13158502928274848  ,  0.6404  ,  0.4559
-0.11640658387226756  ,  0.6728  ,  0.4391
1069
1070
0.0871262746147324  ,  0.8329  ,  0.5498
-0.006926746219699333  ,  0.3209  ,  0.7151
0.053014822943482996  ,  0.3134  ,  0.7191
1071
1072
-0.00296166849246679  ,  1.1428  ,  0.5849
0.025148253825220807  ,  0.1902  ,  0.7918
nan  ,  0.1737  ,  0.7998
1073
1074
-0.007846866892623781  ,  0.2156  ,  0.6742
-0.007843845182136974  ,  0.3064  ,  0.6061
0.03244184733354863  ,  0.2575  ,  0.6413
1075
1076
0.011609979582556771  ,  0.4203  ,  0.5434
-0.05139268703009123  ,  0.2  ,  0.6719
-0.07883903134852688  ,  0.2083  ,  0.6658
1077
1078
-0.017625136835732268  ,  0.4896  ,  0.6638
-0.01751705206628768  ,  0.536  ,  0.5935
0.07526297648884725  ,  0.4814  ,  0.6321
1079
1080
0.05146353249940349  ,  0.5958  ,  0.4584
0.020360967465564282  ,  0.2013  ,  0.5392
0.0075136121559476144  ,  0.1877  ,  0.5448
1081
1082
0.18333491870054297  ,  0.3125  ,  0.7361
-0.104

In [37]:
#replacing nans
from copy import deepcopy
nantup = deepcopy(responses['0.80.9']['worst'][70])
nantup


(nan, 0.3984, 1.546)

In [29]:
pickle.dump(responses,open('./prepped_models/alexnet_sparse/subgraphs/info/subgraph_corellations.pkl','wb'))

In [ ]:
model_act,submodel_act = same_node_subgraph_responses(node,model,submodel, 'small_SPAN', params,batch_size = 20)

In [34]:
###Plotting
import plotly.graph_objs as go
from plotly.graph_objs import *

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [1, 2, 7, 4, 5, 6, 7, 8, 9, 10]
y_upper = [2, 3, 8, 5, 6, 7, 8, 9, 10, 11]
y_lower = [0, 1, 5, 3, 4, 5, 6, 7, 8, 9]


fig = go.Figure([
    go.Scatter(
        x=x,
        y=y,
        line=dict(color='rgb(0,100,80)'),
        mode='lines'
    ),
    go.Scatter(
        x=x+x[::-1], # x, then x reversed
        y=y_upper+y_lower[::-1], # upper, then lower reversed
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    )
])



layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.layout = layout

fig

## subgraph visualizations

In [ ]:
import torch
import sys
sys.path.append('../')
from lucent_edited.optvis import render
from lucent_edited.modelzoo.util import get_model_layers
from torchvision import models

In [ ]:
#visualize  'size' equivalent subgraphs 
model_dis.to('cuda:2')

start = time.time()

version = 'worst'
node = 438
size_model_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/%s_models/%s_.9.9.pt'%(version,str(node)))
size_model = size_model_dict['model']
size_model.to('cuda:0')
name = None
for name, module in size_model.named_modules():
    pass
param_f = lambda: param.image(224,batch=1)
obj =  objectives.neuron(name, 0, batch=0) 
_ = render.render_vis(size_model, obj, param_f, verbose=True, show_inline=True,show_image=True, save_image=False,image_name = 'prepped_models/alexnet_sparse/subgraphs/visualizations/%s_.9.9_target.jpg'%str(node))

print(time.time() - start)

In [ ]:
#visualize subgraphs of different sizes
model_dis.to('cuda:2')

start = time.time()
sizes = ['0.80.9','0.60.8','0.40.7','0.20.6']
node = 200
for size in sizes:
    size_model_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/%s_%s.pt'%(str(node),size))
    size_model = size_model_dict['model']
    size_model.to('cuda:0')
    name = None
    for name, module in size_model.named_modules():
        pass
    param_f = lambda: param.image(224,batch=1)
    obj =  objectives.neuron(name, 0, batch=0) 
    _ = render.render_vis(size_model, obj, param_f, verbose=True, show_inline=True,show_image=True, save_image=False,image_name = 'prepped_models/alexnet_sparse/subgraphs/visualizations/%s_.9.9_target.jpg'%str(node))

print(time.time() - start)

In [ ]:
#get subgraph sizes

#visualize subgraphs of different sizes
model_dis.to('cuda:2')


sizes = ['.9.9','0.80.9','0.60.8','0.40.7','0.20.6']
node = 200
for size in sizes:
    size_model_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/%s_%s.pt'%(str(node),size))
    print(len(size_model_dict['edge_df']))

## Generate graphs of different sizes and types


In [37]:
#extract subgraphs and same size subgraphs of multiple sizes

#run through all subgraphs generating same size subgraphs

import time
start = time.time()



node_thresholds = [.8,.6,.4,.2]

edge_thresholds = [.9,.8,.7,.6]



model_dis.to('cuda:1')

start = time.time()

for node in range(1,313,2):
    
    n_df,e_df=ranksdf_store('small_SPAN', str(node), [],model_dis=model_dis)
    n_df = minmax_normalize_ranks_df(n_df,params)
    for s in range(len(node_thresholds)):
        threshed_n_df,threshed_e_df = hierarchical_accum_threshold(node_thresholds[s],edge_thresholds[s],rank_type,e_df,n_df)
        sub_model = extract_subgraph(model,threshed_n_df,threshed_e_df,params)
        sub_model.to('cuda:1')

        sub_dict = {
                   'model':sub_model,
                   'gen_params':
                        {'node_thresh':node_thresholds[s],
                         'edge_thresh':edge_thresholds[s],
                         'input':'small_SPAN',
                         'output':str(node)
                        },
                    'node_df':threshed_n_df,
                    'edge_df':threshed_e_df
                  }
        sub_dict_path = 'prepped_models/alexnet_sparse/subgraphs/models/%s_%s%s.pt'%(str(node),str(node_thresholds[s]),edge_thresholds[s])
        torch.save(sub_dict,sub_dict_path)
        #make subgraphs
        for version in ['random','worst']:
            for version in ['random','worst']:
                save_path = 'prepped_models/alexnet_sparse/subgraphs/models/%s_models/%s_%s%s.pt'%(version,str(node),str(node_thresholds[s]),edge_thresholds[s])
                n_sizes = get_layer_sizes_from_df(sub_dict['node_df'])
                e_sizes = get_layer_sizes_from_df(sub_dict['edge_df'])
                size_n_df,size_e_df = hierarchical_size_threshold(n_sizes,e_sizes,rank_type,n_df,e_df,selection=version)
                size_model = extract_subgraph(model,size_n_df,size_e_df,params)
                size_dict = {
                   'model':size_model,
                   'gen_params':
                        {'node_sizes':n_sizes,
                         'edge_sizes':e_sizes,
                         'input':'small_SPAN',
                         'output':node,
                         'selection':version,
                         'from_model':sub_dict_path
                        },
                    'node_df':size_n_df,
                    'edge_df':size_e_df
                  }
                torch.save(size_dict,save_path)
                
print(time.time()-start)



Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "1"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_0
1
tensor(-2.0327, device='cuda:2', grad_fn=<SelectBackward>)
target node 1 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_0
1
tensor(-2.6406, device='cuda:2', grad_fn=<SelectBackward>)
target node 1 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_0
1
tensor(-2.7777, device='cuda:2', grad_fn=<SelectBackward>)
target node 1 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_0
1
tensor(-2.4577, device='cuda:2', grad_fn=<SelectBackward>)
target node 1 reached, halted forward pass
batch 4
target reached, breaking model forward pass in features_0
1
tensor(-3.8017, device='cuda:2', grad_fn=<SelectBackward>)
target node 1 reached, halted forward pass
batch 5
target reached, breakin

target reached, breaking model forward pass in features_0
5
tensor(-0.0827, device='cuda:2', grad_fn=<SelectBackward>)
target node 5 reached, halted forward pass
batch 7
target reached, breaking model forward pass in features_0
5
tensor(0.0050, device='cuda:2', grad_fn=<SelectBackward>)
target node 5 reached, halted forward pass
        edge_num   layer_name  layer  out_channel  in_channel  act_rank  \
0              0   features_0      0            0           0  1.312708   
1              1   features_0      0            0           1  1.933793   
2              2   features_0      0            0           2  0.614869   
3              3   features_0      0            1           0  0.494262   
4              4   features_0      0            1           1  1.509352   
...          ...          ...    ...          ...         ...       ...   
250043    250043  features_10      4          255         251  0.000000   
250044    250044  features_10      4          255         252  0.0000

layer: 0
[0 1 2 3 4]
layer: 0
layer: 0
layer: 0
layer: 0
layer: 0
[0 1 2 3 4]
layer: 0
layer: 0
layer: 0
layer: 0
layer: 0
[0 1 2 3 4]
layer: 0
layer: 0
layer: 0
layer: 0
layer: 0
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "11"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_0
11
tensor(-0.0929, device='cuda:2', grad_fn=<SelectBackward>)
target node 11 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_0
11
tensor(-0.0944, device='cuda:2', grad_fn=<SelectBackward>)
target node 11 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_0
11
tensor(-0.0931, device='cuda:2', grad_fn=<SelectBackward>)
target node 11 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_0
11
tensor(-0.0790, device='cuda:2', grad_fn=<SelectBackward>)
target node 11 reached, halted forward pass
batch 4
targ

target reached, breaking model forward pass in features_0
15
tensor(-1.2326, device='cuda:2', grad_fn=<SelectBackward>)
target node 15 reached, halted forward pass
batch 7
target reached, breaking model forward pass in features_0
15
tensor(-1.1745, device='cuda:2', grad_fn=<SelectBackward>)
target node 15 reached, halted forward pass
        edge_num   layer_name  layer  out_channel  in_channel  act_rank  \
0              0   features_0      0            0           0  1.312708   
1              1   features_0      0            0           1  1.933793   
2              2   features_0      0            0           2  0.614869   
3              3   features_0      0            1           0  0.494262   
4              4   features_0      0            1           1  1.509352   
...          ...          ...    ...          ...         ...       ...   
250043    250043  features_10      4          255         251  0.000000   
250044    250044  features_10      4          255         252  0

layer: 0
[0 1 2 3 4]
layer: 0
layer: 0
layer: 0
layer: 0
layer: 0
[0 1 2 3 4]
layer: 0
layer: 0
layer: 0
layer: 0
layer: 0
[0 1 2 3 4]
layer: 0
layer: 0
layer: 0
layer: 0
layer: 0
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "21"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_0
21
tensor(0.2591, device='cuda:2', grad_fn=<SelectBackward>)
target node 21 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_0
21
tensor(-0.1375, device='cuda:2', grad_fn=<SelectBackward>)
target node 21 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_0
21
tensor(-0.7694, device='cuda:2', grad_fn=<SelectBackward>)
target node 21 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_0
21
tensor(1.2105, device='cuda:2', grad_fn=<SelectBackward>)
target node 21 reached, halted forward pass
batch 4
target

...

target node 289 reached, halted forward pass
batch 6
target reached, breaking model forward pass in features_6
33
tensor(-1.8980, device='cuda:2', grad_fn=<SelectBackward>)
target node 289 reached, halted forward pass
batch 7
target reached, breaking model forward pass in features_6
33
tensor(-1.7302, device='cuda:2', grad_fn=<SelectBackward>)
target node 289 reached, halted forward pass
        edge_num   layer_name  layer  out_channel  in_channel  act_rank  \
0              0   features_0      0            0           0  1.312708   
1              1   features_0      0            0           1  1.933793   
2              2   features_0      0            0           2  0.614869   
3              3   features_0      0            1           0  0.494262   
4              4   features_0      0            1           1  1.509352   
...          ...          ...    ...          ...         ...       ...   
250043    250043  features_10      4          255         251  0.000000   
250044   

layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
[0 1 2 3 4]
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
[0 1 2 3 4]
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
[0 1 2 3 4]
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "295"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_6
39
tensor(-1.4682, device='cuda:2', grad_fn=<SelectBackward>)
target node 295 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_6
39
tensor(-1.3462, device='cuda:2', grad_fn=<SelectBackward>)
ta

target node 299 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_6
43
tensor(-2.7014, device='cuda:2', grad_fn=<SelectBackward>)
target node 299 reached, halted forward pass
batch 2
target reached, breaking model forward pass in features_6
43
tensor(-2.1083, device='cuda:2', grad_fn=<SelectBackward>)
target node 299 reached, halted forward pass
batch 3
target reached, breaking model forward pass in features_6
43
tensor(-2.2015, device='cuda:2', grad_fn=<SelectBackward>)
target node 299 reached, halted forward pass
batch 4
target reached, breaking model forward pass in features_6
43
tensor(-3.6569, device='cuda:2', grad_fn=<SelectBackward>)
target node 299 reached, halted forward pass
batch 5
target reached, breaking model forward pass in features_6
43
tensor(-3.5677, device='cuda:2', grad_fn=<SelectBackward>)
target node 299 reached, halted forward pass
batch 6
target reached, breaking model forward pass in features_6
43
tensor(-3.5649, devic

target node 303 reached, halted forward pass
batch 6
target reached, breaking model forward pass in features_6
47
tensor(-3.0609, device='cuda:2', grad_fn=<SelectBackward>)
target node 303 reached, halted forward pass
batch 7
target reached, breaking model forward pass in features_6
47
tensor(-2.2140, device='cuda:2', grad_fn=<SelectBackward>)
target node 303 reached, halted forward pass
        edge_num   layer_name  layer  out_channel  in_channel  act_rank  \
0              0   features_0      0            0           0  1.312708   
1              1   features_0      0            0           1  1.933793   
2              2   features_0      0            0           2  0.614869   
3              3   features_0      0            1           0  0.494262   
4              4   features_0      0            1           1  1.509352   
...          ...          ...    ...          ...         ...       ...   
250043    250043  features_10      4          255         251  0.000000   
250044   

layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
[0 1 2 3 4]
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
[0 1 2 3 4]
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
[0 1 2 3 4]
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
layer: 0
layer: 1
layer: 2
Updating cached rank dfs with small_SPAN
running model to get ranks for "small_SPAN" on target "309"
using device cuda:2
batch 0
target reached, breaking model forward pass in features_6
53
tensor(-0.2609, device='cuda:2', grad_fn=<SelectBackward>)
target node 309 reached, halted forward pass
batch 1
target reached, breaking model forward pass in features_6
53
tensor(0.3588, device='cuda:2', grad_fn=<SelectBackward>)
tar

In [ ]:
random_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/random_models/1000_0.20.6.pt')
worst_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/worst_models/1000_0.20.6.pt')
best_dict = torch.load('prepped_models/alexnet_sparse/subgraphs/models/1000_0.20.6.pt')

In [ ]:
best_dict['model']

In [ ]:
worst_dict['model']


In [ ]:
print(len(random_dict['node_df'].loc[random_dict['node_df']['layer']==2]))
print(len(best_dict['node_df'].loc[best_dict['node_df']['layer']==2]))
print(len(worst_dict['node_df'].loc[worst_dict['node_df']['layer']==2]))

In [ ]:
best_dict['node_df'].loc[best_dict['node_df']['layer']==2].sort_values('node_num_by_layer')

In [ ]:
worst_dict['node_df'].loc[worst_dict['node_df']['layer']==2].sort_values('node_num_by_layer')

In [ ]:
nodeid_2_perlayerid(200,params)